---
title: "Code Contribution and Scientific Authorship"
authors:
  - name: "Eva Maxfield Brown"
    email: evamxb@uw.edu
    orcid: 0000-0003-2564-0373
    affiliations:
      - ref: 1
    attributes:
        corresponding: true
  - name: "Isaac Slaughter"
    orcid: 0000-0002-1911-2374
    affiliations:
      - ref: 1
  - name: "Nicholas Weber"
    orcid: 0000-0002-6008-3763
    affiliations:
      - ref: 1

affiliations:
  - id: 1
    name: University of Washington Information School

abstract: |
  Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur eget porta erat. Morbi consectetur est vel gravida pretium. Suspendisse ut dui eu ante cursus gravida non sed sem. Nullam sapien tellus, commodo id velit id, eleifend volutpat quam. Phasellus mauris velit, dapibus finibus elementum vel, pulvinar non tellus. Nunc pellentesque pretium diam, quis maximus dolor faucibus id. Nunc convallis sodales ante, ut ullamcorper est egestas vitae. Nam sit amet enim ultrices, ultrices elit pulvinar, volutpat risus.

## Basics
bibliography: main.bib

## Number sections (required for section cross ref)
number-sections: true

## Citation Style Language
# See https://github.com/citation-style-language/styles for more options
# We default to PNAS (Proceedings of the National Academy of Sciences)
# csl: support/acm-proceedings.csl

## Specific for target format
format:
  html:
    standalone: true
    embed-resources: true
    toc: true
    toc-location: left
    reference-location: margin
    citation-location: document
    execute:
      echo: false
      warning: false

  elsevier-pdf:
    toc: false
    execute:
      echo: false
      warning: false
    include-in-header:  
      - text: |
          \usepackage{booktabs}    % For professional-looking tables
          \usepackage{multirow}    % For merged cells
          \usepackage{siunitx}     % For number alignment
          \usepackage[table]{xcolor}  % The 'table' option is crucial for \rowcolors to work
          \usepackage{float}
          \floatplacement{table}{H}
          \usepackage{lineno}
          \linenumbers
        
    journal:
      cite-style: authoryear
      name: "in-progress"
      model: "3p"
      layout: "onecolumn"

    # Word count
    citeproc: false
    filters:
      - at: pre-quarto
        path: _extensions/andrewheiss/wordcount/citeproc.lua
      - at: pre-quarto
        path: _extensions/andrewheiss/wordcount/wordcount.lua
---


# Introduction


In [ ]:
#| output: false

from collections import OrderedDict
from datetime import datetime
from io import StringIO
import os
from pathlib import Path

import colormaps as cmaps
import IPython.display
import matplotlib.pyplot as plt
import numpy as np
import opinionated  # noqa
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sci_soft_models.dev_author_em import data as sci_soft_models_data
from scipy.stats import (
    binomtest,
    chi2_contingency,
    pearsonr,
)
from statsmodels.stats.multitest import multipletests
from sqlalchemy import text, create_engine

from rs_graph.db import models as db_models
from rs_graph.db.constants import PROD_DATABASE_FILEPATH

# Get db engine for production database
db_conn = create_engine(f"sqlite:///{PROD_DATABASE_FILEPATH}")

# Set seaborn style
plt.style.use("opinionated_rc")
sns.set_palette(
    cmaps.bold[2:]._colors.tolist(),
)

# Get or set USE_SAMPLE
if "QSS_CODE_AUTHORSHIP_USE_SAMPLE" in os.environ:
    USE_SAMPLE = bool(int(os.environ["QSS_CODE_AUTHORSHIP_USE_SAMPLE"]))
else:
    USE_SAMPLE = True
# USE_SAMPLE = False

In [ ]:
def read_table(table: str) -> pd.DataFrame:
    return pd.read_sql(text(f"SELECT * FROM {table}"), db_conn)


# Read all data from database
doc_repo_links = read_table(db_models.DocumentRepositoryLink.__tablename__)
researchers = read_table(db_models.Researcher.__tablename__)
devs = read_table(db_models.DeveloperAccount.__tablename__)
documents = read_table(db_models.Document.__tablename__)
document_contributors = read_table(db_models.DocumentContributor.__tablename__)
repositories = read_table(db_models.Repository.__tablename__)
repository_contributors = read_table(db_models.RepositoryContributor.__tablename__)
topics = read_table(db_models.Topic.__tablename__)
document_topics = read_table(db_models.DocumentTopic.__tablename__)
dataset_sources = read_table(db_models.DatasetSource.__tablename__)
researcher_dev_links = read_table(
    db_models.ResearcherDeveloperAccountLink.__tablename__
)
document_alternate_dois = read_table(
    db_models.DocumentAlternateDOI.__tablename__
)

# Drop all "updated_datetime" and "created_datetime" columns
for df in [
    doc_repo_links,
    researchers,
    devs,
    documents,
    document_contributors,
    repositories,
    repository_contributors,
    topics,
    document_topics,
    dataset_sources,
    researcher_dev_links,
]:
    df.drop(columns=["updated_datetime", "created_datetime"], inplace=True)

# Specifically drop doc_repo_links "id" column
# It isn't used and will get in the way later when we do a lot of joins
doc_repo_links.drop(columns=["id"], inplace=True)

# Construct reduced doc_repo_links
original_doc_repo_links_len = len(doc_repo_links)
doc_repo_links = doc_repo_links.drop_duplicates(subset=["document_id"], keep=False)
doc_repo_links = doc_repo_links.drop_duplicates(subset=["repository_id"], keep=False)

# Reduce other tables to only documents / repositories in the updated doc_repo_links
documents = documents[documents["id"].isin(doc_repo_links["document_id"])]
repositories = repositories[repositories["id"].isin(doc_repo_links["repository_id"])]
document_contributors = document_contributors[
    document_contributors["document_id"].isin(documents["id"])
]
repository_contributors = repository_contributors[
    repository_contributors["repository_id"].isin(repositories["id"])
]
document_topics = document_topics[document_topics["document_id"].isin(documents["id"])]

# Reduce researchers and devs to only those in the
# updated document_contributors and repository_contributors
researchers = researchers[
    researchers["id"].isin(document_contributors["researcher_id"])
]
devs = devs[devs["id"].isin(repository_contributors["developer_account_id"])]
researcher_dev_links = researcher_dev_links[
    (
        researcher_dev_links["researcher_id"].isin(researchers["id"])
        & researcher_dev_links["developer_account_id"].isin(devs["id"])
    )
]

# Sort document topics and keep first
document_topics = document_topics.sort_values("score", ascending=False)
document_topics = document_topics.drop_duplicates(subset=["document_id"], keep="first")

# Create document, document topic merged table
merged_document_topics = pd.merge(
    document_topics, topics, left_on="topic_id", right_on="id"
)

# Create basic merged tables
merged_document_contributor_doc_repo_links = pd.merge(
    document_contributors, doc_repo_links, left_on="document_id", right_on="document_id"
)
merged_repository_contributor_doc_repo_links = pd.merge(
    repository_contributors,
    doc_repo_links,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for data sources
data_source_stats = []
for _, data_source in dataset_sources.iterrows():
    # Get total article-repo pairs
    data_source_stats.append(
        {
            "data_source": data_source["name"],
            "n_article_repo_pairs": len(
                doc_repo_links[doc_repo_links["dataset_source_id"] == data_source["id"]]
            ),
            "n_authors": merged_document_contributor_doc_repo_links.loc[
                merged_document_contributor_doc_repo_links["dataset_source_id"]
                == data_source["id"]
            ]["researcher_id"].nunique(),
            "n_devs": merged_repository_contributor_doc_repo_links.loc[
                merged_repository_contributor_doc_repo_links["dataset_source_id"]
                == data_source["id"]
            ]["developer_account_id"].nunique(),
        }
    )

# Create topic merged tables
merged_doc_repo_links_topics = pd.merge(
    doc_repo_links, document_topics, left_on="document_id", right_on="document_id"
).merge(topics, left_on="topic_id", right_on="id")
merged_doc_repo_links_topics_document_contributors = pd.merge(
    merged_doc_repo_links_topics,
    document_contributors,
    left_on="document_id",
    right_on="document_id",
)
merged_doc_repo_links_topics_repository_contributors = pd.merge(
    merged_doc_repo_links_topics,
    repository_contributors,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for domains
domain_stats = []
for domain in merged_doc_repo_links_topics.domain_name.unique():
    # Get total article-repo pairs
    domain_stats.append(
        {
            "domain": domain,
            "n_article_repo_pairs": len(
                merged_doc_repo_links_topics[
                    merged_doc_repo_links_topics["domain_name"] == domain
                ]
            ),
            "n_authors": merged_doc_repo_links_topics_document_contributors.loc[
                merged_doc_repo_links_topics_document_contributors["domain_name"]
                == domain
            ]["researcher_id"].nunique(),
            "n_devs": merged_doc_repo_links_topics_repository_contributors.loc[
                merged_doc_repo_links_topics_repository_contributors["domain_name"]
                == domain
            ]["developer_account_id"].nunique(),
        }
    )

# Create document merged tables
merged_doc_repo_links_documents = pd.merge(
    doc_repo_links, documents, left_on="document_id", right_on="id"
)
merged_doc_repo_links_documents_document_contributors = pd.merge(
    merged_doc_repo_links_documents,
    document_contributors,
    left_on="document_id",
    right_on="document_id",
)
merged_doc_repo_links_documents_repository_contributors = pd.merge(
    merged_doc_repo_links_documents,
    repository_contributors,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for document types
# This isn't a standard data pull
# In short:
# - pairs from PLOS are "research articles"
# - pairs from JOSS are "software articles"
# - pairs from SoftwareX are "software articles"
# - pairs from Papers with Code / ArXiv are "pre-prints"
#   UNLESS they have been published in a journal
# All of those should be easy to assert / apply a label to with the exception
# of Papers with Code / ArXiv pre-prints that have been published in a journal
# In that case, we need to look at the existing document type in the database
# If the document type is "preprint" use preprint, otherwise, if it's anything else,
# use "research article"

# Create a "reduced_doc_types" dataframe with document_id and "reduced_doc_type"
# columns
reduced_doc_types_rows = []
# We can use the "reduced_doc_types" dataframe to calculate the stats

# Iter over data sources even though we are looking for doc types
for _, data_source in dataset_sources.iterrows():
    # Get total article-repo pairs
    doc_type = None
    if data_source["name"] in ["plos", "joss", "softwarex"]:
        if data_source["name"] == "plos":
            doc_type = "research article"
        else:
            doc_type = "software article"

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": doc_type}
                for doc_id in doc_repo_links[
                    (doc_repo_links["dataset_source_id"] == data_source["id"])
                ]["document_id"]
            ]
        )

    # Handle PwC
    else:
        # Get preprint pairs
        preprint_pairs = merged_doc_repo_links_documents[
            (merged_doc_repo_links_documents["dataset_source_id"] == data_source["id"])
            & (merged_doc_repo_links_documents["document_type"] == "preprint")
        ]

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": "preprint"}
                for doc_id in preprint_pairs["document_id"]
            ]
        )

        # Get research article pairs
        # This is the same just inverted to != "preprint"
        research_article_pairs = merged_doc_repo_links_documents[
            (merged_doc_repo_links_documents["dataset_source_id"] == data_source["id"])
            & (merged_doc_repo_links_documents["document_type"] != "preprint")
        ]

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": "research article"}
                for doc_id in research_article_pairs["document_id"]
            ]
        )

# Create reduced_doc_types dataframe
reduced_doc_types = pd.DataFrame(reduced_doc_types_rows)

In [ ]:
# Precompute DOI resolution statistics as that will be referenced
# in the Data and Methods section

# Groupby "name" and get percent have resolved dois
per_dataset_doi_resolution = {}
for dataset_source, group in doc_repo_links.merge(
    dataset_sources,
    left_on="dataset_source_id",
    right_on="id",
).groupby("name"):
    # Get unique articles in this group
    unique_group_articles = group.drop_duplicates(subset=["document_id"])

    # Get the number of articles from this dataset
    n_articles_in_group = len(unique_group_articles)
    
    # Get the number of articles with an alternate doi in this group
    # Where the alternate doi isn't the same as the document doi
    group_document_alternate_dois = document_alternate_dois[
        (document_alternate_dois["document_id"].isin(unique_group_articles["document_id"]))
        & (~document_alternate_dois["doi"].str.lower().isin(documents["doi"].str.lower()))
    ]
    n_articles_with_alt_doi = len(group_document_alternate_dois)

    # Add to results
    per_dataset_doi_resolution[dataset_source] = n_articles_with_alt_doi / n_articles_in_group

# Get overall doi resolution
n_articles = len(documents)

# Get the number of articles with an alternate doi
# Where the alternate doi isn't the same as the document doi
unique_document_alternate_dois = document_alternate_dois[
    ~document_alternate_dois["doi"].str.lower().isin(documents["doi"].str.lower())
]
n_articles_with_alt_doi = len(unique_document_alternate_dois)

# Compute overall doi resolution
overall_doi_resolution = n_articles_with_alt_doi / n_articles

The communal sharing of new knowledge versus individual recognition for discovery is an essential tension in science \[\@Kuhn77\]. The resolution of this tension is perhaps most pointed in the publishing of articles where new scientific claims are made and credit is imperfectly allocated to the authors. The quantitative study of science has historically used the published record of scientific claims as its primary data source - benefiting greatly from standardization and curation of bibliographic information from indexes like the Scientific Citation Index [@Bergstrom et al].

Contemporary scientific research is increasingly collaborative, complex, and computationally dependent. These innovations present new challenges to the quantitative study of science, as well as new opportunities to understand how new scientific knowledge is produced and credit is allocated. There has been a significant literature focused on the growth of author lists, and the increasing role of technicians that support complex scientific projects. However, there is surprisingly little research that focuses on alternative data sources that might ...

For example, emergence of public code repositories alongside published research provides a unique opportunity to study this disconnect. Source code repositories maintain detailed records of who contributes what code and when, providing a window into the patterns of software development in scientific research. Like SCI - these data are accessible and standized - enabling robust quantitative research

In this article we examine this tension anew -

Using transaction histories from these repositories, we develop a predictive model that enables systematic matching between the authors of a scientific article and the developer accounts of related source code repositories. We identify several patterns that distinguish code contributions from scientific recognition by applying our predictive model across almost 140,000 paired research articles and repositories. We show that nearly 30% (n=X) of articles have non-author code-contributors - individuals who may have helped create the software but received no formal authorship credit. We find that code-contributing authors are associated with a modest increase in article-level impact metrics (\~5.1% increase in citations per code-contributing author). However, these effects become statistically non-significant when controlling for domain, article type, and open access status. First authors are significantly more likely to be code contributors than other author positions across all conditions tested. We also find a negative relationship between coding frequency and scholarly impact: Authors who contribute code more frequently show progressively lower h-indices than their non-coding peers, a pattern that persists when controlling for publication count, and author's most common author position, domain, and article type.

The primary contributions of this article are: (1) a predictive model for matching authors with developer account information that addresses challenges in identity resolution across platforms[^1]; (2) a dataset of linked authors and developers for \~140,000 article-repository pairs, providing a resource for analyzing scientific software development patterns[^2]; and (3) analyses that reveal insights into the software development dynamics of research teams, including patterns of recognition, impact, and career implications for code contributors. These contributions provide evidence for the ongoing discussions about scientific recognition systems and raise questions about aligning institutional incentives with the spectrum of contributions that drive modern scientific progress.

[^1]: Our predictive model is made available via Huggingface at <https://huggingface.co/evamxb/dev-author-em-clf> or via our Python package, sci-soft-models (<https://github.com/evamaxfield/sci-soft-models/>).

[^2]: Our dataset is made available via Harvard Dataverse at <https://doi.org/10.7910/DVN/KPYVI1>. Portions of the dataset are restricted to keep researcher and developer information private. The full dataset is available upon request via the Harvard Dataverse. For details on interacting with this dataset, see our documentation at <https://github.com/evamaxfield/rs-graph>.

The remainder of this paper proceeds as follows. First, we review related work regarding scientific software development and the recognition of code contributors in scientific credit systems. In addition, we introduce the specific hypotheses that guide our investigation. Next, we detail our data and methods, describing how we created a dataset of linked article-repository pairs, trained and evaluated our predictive model for entity matching, and applied our model across each article-repository pair. We then present our data analysis, focusing on article-level dynamics before moving to individual-level patterns, formally accepting or rejecting each hypothesis based on our findings. We conclude by discussing the results, limitations of our work, and areas for future improvement.

# Background {#sec-background}

## Research Team Composition and Scientific Impact

Except for foundational methodological shifts, scientific recognition systems have favored experimental and theoretical contributions more than their methodological counterparts, with experimental and theoretical articles receiving higher citation rates [@Aksnes2006CitationRA; @LIU2023101432; @Chen2024ExploringSC]. However, computational methods have transformed researchers' work across all scientific disciplines. Modern scientific endeavors increasingly depend on sophisticated computational approaches, whether for processing large-scale experimental data, running complex simulations, or developing new methodological tools [@jin2015significance; @hampton2013big; @edwards2013knowledge; @mayernik2017assessing; @hasselbring2024researchsoftwarecategories].

The computational evolution in scientific practice intersects with established findings about team dynamics in both research and software development. Prior research has shown that larger and more diverse teams typically produce higher-impact scientific work [@Franceschet2010TheEO; @Larivire2014TeamSM], while in software engineering, larger development teams tend to create more reliable software with fewer defects [@Wyss2023NothingBut]. These findings suggest that team size may be particularly important in scientific software development, where technical robustness and scientific innovation are crucial.

The unique characteristics of scientific software development - including implementing novel algorithms, requiring deep domain knowledge, and an increased emphasis on reproducibility [@muna2016astropyproblem; @Howison2013IncentivesAI] - make team composition especially relevant. Larger development teams may enhance scientific impact through multiple mechanisms: they can produce more robust and generalizable software tools for methodological contributions while enabling more sophisticated computational analyses and larger-scale data processing for experimental work. Given these patterns in team dynamics, software development practices, and the computational transformation of scientific work, we propose:

*H1: The number of individuals contributing code to a publication's associated repository positively correlates with the article's citation count.*

## Author Roles and Technical Contributions

Author positions in scientific publications signal specific roles and responsibilities, a relationship extensively studied through contribution role taxonomies like CRediT [@lariviere2016contributorship]. These studies reveal that first authors and corresponding authors, while occasionally the same individual [@chinchilla2022relationship], take on distinct responsibilities. Analyses of contribution patterns consistently show that software development, data analysis, and visualization tasks typically fall to first authors [@lariviere2016contributorship; @Jnior2016PatternsOA; @Larivire2020InvestigatingTD; @sauermann2017authorship]. Meanwhile, corresponding authors, whether or not they are also first authors, often maintain responsibility for research artifacts' long-term sustainability and reuse, which we believe may include the maintenance and documentation of software tools.

Contribution records from source code repositories provide a unique method to verify these established contribution patterns. Given prior findings about the distribution of technical responsibilities within research teams, we expect these repository records to reflect similar patterns of engagement with software development:

*H2a: First authors have higher code contribution rates than authors in other positions.*

*H2b: Corresponding authors have higher code contribution rates than non-corresponding authors.*

## Code Contribution and Individual Scientific Impact

Despite the increasingly central role of software in science, researchers who develop scientific software face persistent challenges in receiving formal scientific recognition for their contributions. Prior work has shown that software developers in research settings are often relegated to acknowledgment sections rather than receiving authorship credit, even when their technical contributions are fundamental to the research [@Carver2022ASO; @olivier_philippe_2019_2585783].

The challenge of recognition is compounded by inconsistent practices in software citation. While researchers have tried to standardize software citation, actual citation practices remain highly variable across fields and journals [@Lamprecht2020TowardsFP; @Katz2020RecognizingTV; @Smith2016SoftwareCP]. This variability challenges researchers who maintain and update existing software packages. While creating entirely new software may lead to dedicated publications and citations, the ongoing work of maintaining, debugging, and extending existing software - often crucial for scientific progress - typically generates less visible scientific credit [@howison2011incentives; @Howison2013IncentivesAI].

These structural challenges in recognizing and citing software contributions suggest a potential misalignment between technical contributions and traditional scientific impact metrics. When researchers dedicate significant time to software development and maintenance, conventional bibliometric measures may not fully capture their contributions, regardless of the software's importance to the field. Whether through attribution practices that favor acknowledgments over authorship or citation patterns that undervalue maintenance work, multiple mechanisms could lead to lower traditional impact metrics for active code contributors. Based on these patterns in software recognition and citation, we hypothesize:

*H3: The frequency with which individual researchers contribute code to their research projects is negatively correlated with their h-index.*

# Data and Methods

Our analysis examines the relationship between software development contributions and scientific credit attribution through a three-step process: (1) building a dataset of linked scientific articles and code repositories, (2) developing a predictive model to match article authors with developer accounts, and (3) analyzing patterns in these relationships.

Our dataset integrates article-repository pairs from four sources, each with explicit mechanisms for code sharing: the Journal of Open Source Software (JOSS) and SoftwareX require code repositories as part of publication, Papers with Code directly connects preprints with implementations, and Public Library of Science (PLOS) articles include mandatory data and code availability statements that we mined for repository links. We focused exclusively on GitHub-hosted repositories, which represent the predominant platform for scientific software sharing [@Cao2023TheRO; @escamilla2022riseofgithub]. For each article-repository pair, we resolved DOIs via Semantic Scholar to ensure we captured the latest version of each publication, extracted publication metadata and author metrics through OpenAlex, and collected repository information via the GitHub API This created a comprehensive dataset with information about both the scientific and software development aspects of each project. All data was collected between October and November 2024.


In [ ]:
exp_results = sci_soft_models_data.load_experimental_training_results()

# Remove the "time_pred" and "epoch_val" columns
exp_results = exp_results.drop(columns=["time_pred", "epoch_val"])

# Round accuracy, precision, recall, and f1 to 3 decimal places
exp_results = exp_results.round(
    {
        "accuracy": 3,
        "precision": 3,
        "recall": 3,
        "f1": 3,
    }
)

# Replace "no-optional-data" with "n/a"
exp_results["fieldset"] = exp_results["fieldset"].replace(
    "no-optional-data", "n/a"
)

# Str replace references to "dev_" in "fieldset" column
exp_results["fieldset"] = exp_results["fieldset"].str.replace("dev_", "")

# Str replace "_" with " " in "fieldset" column
exp_results["fieldset"] = exp_results["fieldset"].str.replace("_", " ")

# Str split fieldset values on "-" and rejoin with ", "
exp_results["fieldset"] = exp_results["fieldset"].str.split("-").apply(
    lambda x: ", ".join(x)
)

# Rename fieldset column to "Optional Features"
exp_results = exp_results.rename(columns={"fieldset": "Optional Feats."})

# Capitalize column names
exp_results.columns = exp_results.columns.str.title()

# Get best model f1 and get the accuracy, precision, recall, and f1 for that model
best_model_idx = exp_results["F1"].idxmax()
best_model_acc = exp_results.loc[best_model_idx, "Accuracy"]
best_model_prec = exp_results.loc[best_model_idx, "Precision"]
best_model_rec = exp_results.loc[best_model_idx, "Recall"]
best_model_f1 = exp_results.loc[best_model_idx, "F1"]

To match article authors with repository developer accounts, we developed a machine learning approach using transformer-based architectures. Entity matching presents unique challenges, as exact name or email matching is insufficient due to formatting variations and incomplete information (e.g., "J. Doe" vs. "Jane Doe" in publications, or use of institutional versus personal email addresses). While exact matching fails, there is typically high semantic overlap between author information and developer account details that our model can leverage. We created a gold-standard dataset of 3,000 annotated author-developer account pairs from JOSS publications, where two independent reviewers classified each pair as matching or non-matching. After systematic evaluation of three transformer architectures with various feature combinations, our optimal model (fine-tuned DeBERTa-v3-base including developer account username and display name in the training data) achieved an F1 score of `{python} float(best_model_f1)`, with `{python} float(best_model_prec)` precision and `{python} float(best_model_rec)` recall (detailed comparison available in the Appendix).

Applying our model across all article-repository pairs yielded a large-scale dataset linking scientific authors and code contributors. As shown in @tbl-rs-graph-overall-counts, our full, unfiltered dataset contains approximately 140,000 unique article-repository pairs spanning multiple domains and publication types, with nearly 300,000 distinct authors and more than 150,000 developer accounts. From these, we identified almost 110,000 author-developer account relationships, creating a unique resource for investigating code contribution patterns in scientific teams. This dataset enables systematic examination of how software development work relates to scientific recognition and career trajectories. Both the dataset (<https://doi.org/10.7910/DVN/KPYVI1>) and model (<https://huggingface.co/evamxb/dev-author-em-clf>) are made publicly available to support further research.


In [ ]:
# Compute stats
doc_type_stats = reduced_doc_types.groupby("reduced_doc_type").apply(
    lambda x: {
        "doc_type": x.name,
        "n_article_repo_pairs": len(x),
        "n_authors": merged_doc_repo_links_documents_document_contributors.loc[
            merged_doc_repo_links_documents_document_contributors["document_id"].isin(
                x["document_id"]
            )
        ]["researcher_id"].nunique(),
        "n_devs": merged_doc_repo_links_documents_repository_contributors.loc[
            merged_doc_repo_links_documents_repository_contributors["document_id"].isin(
                x["document_id"]
            )
        ]["developer_account_id"].nunique(),
    },
    include_groups=False,
)

# Compute stats for access status
access_stats = []
for access_status_int, access_status_name in [
    (0, "Closed"),
    (1, "Open"),
]:
    # Get total article-repo pairs
    access_stats.append(
        {
            "access_status": access_status_name,
            "n_article_repo_pairs": len(
                merged_doc_repo_links_documents[
                    merged_doc_repo_links_documents["is_open_access"]
                    == access_status_int
                ]
            ),
            "n_authors": merged_doc_repo_links_documents_document_contributors.loc[
                merged_doc_repo_links_documents_document_contributors["is_open_access"]
                == access_status_int
            ]["researcher_id"].nunique(),
            "n_devs": merged_doc_repo_links_documents_repository_contributors.loc[
                merged_doc_repo_links_documents_repository_contributors[
                    "is_open_access"
                ]
                == access_status_int
            ]["developer_account_id"].nunique(),
        }
    )

# Compute totals
total_article_repo_pairs = len(doc_repo_links)
total_authors = merged_document_contributor_doc_repo_links["researcher_id"].nunique()
total_devs = merged_repository_contributor_doc_repo_links[
    "developer_account_id"
].nunique()

::: {.content-visible when-format="html"}

In [ ]:
def _get_html_for_counts_table(
    domain_stats,
    doc_type_stats,
    access_stats,
    data_source_stats,
    total_article_repo_pairs,
    total_authors,
    total_devs,
):
    # Construct multi-row span HTML table
    # Columns should be: "n_article_repo_pairs", "n_authors", "n_devs"
    # Rows should be:
    # "By Data Source", "By Domain", "By Document Type", "By Access Status", and "Total"

    # HTML templates
    stats_piece_inital_row_template = """
    <tr>
    <td rowspan="{n_rows}">{row_name}</td>
    <td>{value_name}</td>
    <td>{article_repo_pairs}</td>
    <td>{authors}</td>
    <td>{devs}</td>
    </tr>
    """.strip()

    stats_piece_subsequent_row_template = """
    <tr>
    <td>{value_name}</td>
    <td>{article_repo_pairs}</td>
    <td>{authors}</td>
    <td>{devs}</td>
    </tr>
    """.strip()

    # Iter over stats portions (and total)
    stats_portions_html = []
    for stats_portion, stats_name, value_key in [
        (domain_stats, "<b>By Domain</b>", "domain"),
        (doc_type_stats, "<b>By Document Type</b>", "doc_type"),
        (access_stats, "<b>By Access Status</b>", "access_status"),
        (data_source_stats, "<b>By Data Source</b>", "data_source"),
        (
            [
                {
                    "empty": "",
                    "n_article_repo_pairs": f"<b>{total_article_repo_pairs}</b>",
                    "n_authors": f"<b>{total_authors}</b>",
                    "n_devs": f"<b>{total_devs}</b>",
                }
            ],
            "<b>Total</b>",
            "empty",
        ),
    ]:
        # Order by value_key
        if value_key != "empty":
            stats_portion = sorted(
                stats_portion, key=lambda x: x[value_key]
            )

        stats_portion_html = []
        for i, stats_piece in enumerate(stats_portion):
            if i == 0:
                stats_portion_html.append(
                    stats_piece_inital_row_template.format(
                        n_rows=len(stats_portion),
                        row_name=stats_name,
                        value_name=stats_piece[value_key],
                        article_repo_pairs=stats_piece["n_article_repo_pairs"],
                        authors=stats_piece["n_authors"],
                        devs=stats_piece["n_devs"],
                    )
                )
            else:
                stats_portion_html.append(
                    stats_piece_subsequent_row_template.format(
                        value_name=stats_piece[value_key],
                        article_repo_pairs=stats_piece["n_article_repo_pairs"],
                        authors=stats_piece["n_authors"],
                        devs=stats_piece["n_devs"],
                    )
                )

        stats_portions_html.append("\n".join(stats_portion_html))

    # Concat and wrap in table
    stats_table_html = f"""
    <table>
    <tr>
        <th><b>Category</b></th>
        <th><b>Subset</b></th>
        <th><b>Article-Repository Pairs</b></th>
        <th><b>Authors</b></th>
        <th><b>Developers</b></th>
    </tr>
    {" ".join(stats_portions_html)}
    </table>
    """.strip()

    return stats_table_html

In [ ]:
#| label: tbl-rs-graph-overall-counts
#| tbl-cap: Counts of Article-Repository Pairs, Authors, and Developers by Data Sources, Domains, Document Types, and Access Status in the Full, Unfiltered Dataset.

stats_table_html = _get_html_for_counts_table(
    domain_stats=domain_stats,
    doc_type_stats=doc_type_stats,
    access_stats=access_stats,
    data_source_stats=data_source_stats,
    total_article_repo_pairs=total_article_repo_pairs,
    total_authors=total_authors,
    total_devs=total_devs,
)

IPython.display.HTML(stats_table_html)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
def _get_latex_for_counts_table(
    caption,
    label,
    domain_stats,
    doc_type_stats,
    access_stats,
    data_source_stats,
    total_article_repo_pairs,
    total_authors,
    total_devs,
):
    # LaTeX templates
    stats_piece_initial_row_template = """    \\multirow{{{n_rows}}}{{*}}{{\\textbf{{{row_name}}}}} & \\cellcolor{{gray!10}}{value_name} & \\cellcolor{{gray!10}}{article_repo_pairs} & \\cellcolor{{gray!10}}{authors} & \\cellcolor{{gray!10}}{devs} \\\\"""

    stats_piece_subsequent_row_template = """    & {value_name} & {article_repo_pairs} & {authors} & {devs} \\\\"""

    # Table header and footer templates
    table_header_start = """\\begin{{table}}
    \\centering
    \\small
    \\caption{{{caption}}}
    \\label{{{label}}}
    \\begin{{tabular}}{{llrrr}}
    \\toprule
    \\textbf{{Category}} & \\textbf{{Subset}} & \\textbf{{Article-Repository Pairs}} & \\textbf{{Authors}} & \\textbf{{Developers}} \\\\
    \\midrule""".format(
        caption=caption,
        label=label,
    )

    table_footer = """\\bottomrule
    \\end{tabular}
    \\end{table}"""

    # Generate table content
    stats_portions_latex = []
    for stats_portion, stats_name, value_key in [
        (domain_stats, "By Domain", "domain"),
        (doc_type_stats, "By Document Type", "doc_type"),
        (access_stats, "By Access Status", "access_status"),
        (data_source_stats, "By Data Source", "data_source"),
        (
            [
                {
                    "empty": "",
                    "n_article_repo_pairs": total_article_repo_pairs,
                    "n_authors": total_authors,
                    "n_devs": total_devs,
                }
            ],
            "Total",
            "empty",
        ),
    ]:
        # Order by value_key
        if value_key != "empty":
            stats_portion = sorted(
                stats_portion, key=lambda x: x[value_key]
            )

        stats_portion_latex = []
        if stats_name == "Total":
            stats_portion_latex.append(
                f"    \\textbf{{Total}} & & \\textbf{{{stats_portion[0]['n_article_repo_pairs']}}} & \\textbf{{{stats_portion[0]['n_authors']}}} & \\textbf{{{stats_portion[0]['n_devs']}}} \\\\"
            )
        else:
            # Add the first row with the category label and colored cells
            first_piece = stats_portion[0]
            stats_portion_latex.append(
                stats_piece_initial_row_template.format(
                    n_rows=len(stats_portion),
                    row_name=stats_name,
                    value_name=first_piece[value_key],
                    article_repo_pairs=first_piece["n_article_repo_pairs"],
                    authors=first_piece["n_authors"],
                    devs=first_piece["n_devs"],
                ).rstrip()
            )
            
            # Add subsequent rows with alternating colors
            for i, stats_piece in enumerate(stats_portion[1:]):
                color_cmd = "\\cellcolor{gray!10}" if i % 2 == 1 else ""
                stats_portion_latex.append(
                    f"    & {color_cmd}{stats_piece[value_key]} & {color_cmd}{stats_piece['n_article_repo_pairs']} & {color_cmd}{stats_piece['n_authors']} & {color_cmd}{stats_piece['n_devs']} \\\\"
                )

        section_latex = "\n".join(stats_portion_latex)
        if stats_name != "Total":
            section_latex += "\\midrule"
        stats_portions_latex.append(section_latex)

    # Combine all parts
    stats_table_latex = f"""{table_header_start}
    {"\n".join(stats_portions_latex)}
    {table_footer}"""

    return stats_table_latex

In [ ]:
stats_table_latex = _get_latex_for_counts_table(
    caption="Counts of Article-Repository Pairs, Authors, and Developers by Data Sources, Domains, Document Types, and Access Status in the Full, Unfiltered Dataset.",
    label="tbl-rs-graph-overall-counts",
    domain_stats=domain_stats,
    doc_type_stats=doc_type_stats,
    access_stats=access_stats,
    data_source_stats=data_source_stats,
    total_article_repo_pairs=total_article_repo_pairs,
    total_authors=total_authors,
    total_devs=total_devs,
)

IPython.display.Latex(stats_table_latex)

:::

# Analysis of Code Contributor Authorship and Development Dynamics of Research Teams

## Software Development Dynamics Within Research Teams


In [ ]:
# Create subset documents
docs_w_1_citation = documents.loc[documents["cited_by_count"] >= 1].copy()

# Take sample?
if USE_SAMPLE:
    docs_w_1_citation = docs_w_1_citation.sample(frac=0.02, random_state=12)

# Subset to only certain columns
docs_w_1_citation = docs_w_1_citation[
    [
        "id",
        "publication_date",
        "cited_by_count",
        "fwci",
        "is_open_access",
    ]
]

# Rename id to document_id
docs_w_1_citation = docs_w_1_citation.rename(columns={"id": "document_id"})

# Merge repository id in
docs_w_1_citation = docs_w_1_citation.merge(
    doc_repo_links[[
        "document_id",
        "repository_id",
    ]],
    left_on="document_id",
    right_on="document_id",
)

# Merge in document details (domain, document type)
docs_w_1_citation = (
    docs_w_1_citation.merge(
        document_topics[["document_id", "topic_id"]],
        left_on="document_id",
        right_on="document_id",
    )
    .merge(
        repositories[["id", "creation_datetime", "last_pushed_datetime"]],
        left_on="repository_id",
        right_on="id",
    )
    .drop(
        columns=["id"],
    )
    .merge(
        topics[["id", "domain_name"]],
        left_on="topic_id",
        right_on="id",
    )
    .drop(
        columns=["id", "topic_id"],
    )
    .merge(
        reduced_doc_types,
        left_on="document_id",
        right_on="document_id",
    )
    .rename(
        columns={
            "domain_name": "domain",
            "reduced_doc_type": "article_type",
        }
    )
)

# Drop any documents that have more than one repository (and vice versa)
docs_w_1_citation = docs_w_1_citation.drop_duplicates(
    subset=["document_id"], keep=False
)
docs_w_1_citation = docs_w_1_citation.drop_duplicates(
    subset=["repository_id"], keep=False
)

# Iter over articles and get the team composition info
team_composition_rows = []
relationships_removed_from_confidence_thresh = set()
for _, row in docs_w_1_citation.iterrows():
    # Get a boolean value for "no pushes after publication"
    repo_details = repositories.loc[repositories["id"] == row["repository_id"]].iloc[0]

    # Get the number of authors
    author_ids = document_contributors.loc[
        document_contributors["document_id"] == row["document_id"]
    ]["researcher_id"].unique()
    n_authors = len(author_ids)

    # Get the number of devs
    dev_ids = repository_contributors.loc[
        repository_contributors["repository_id"] == row["repository_id"]
    ]["developer_account_id"].unique()
    n_devs = len(dev_ids)

    # Get the set of researcher_dev_links for the authors
    author_dev_links = researcher_dev_links.loc[
        researcher_dev_links["researcher_id"].isin(author_ids)
    ].sort_values("predictive_model_confidence", ascending=False)

    # Drop duplicates by developer_account_id (keeping first)
    # as we may have accidently matched the same dev to the multiple authors
    author_dev_links = author_dev_links.drop_duplicates(
        subset=["developer_account_id"],
        keep="first",
    )

    # Drop any author dev links that have less than 97% confidence
    author_dev_links_filtered = author_dev_links.loc[
        author_dev_links["predictive_model_confidence"] >= 0.97
    ]
    for _, relationship_row in author_dev_links.loc[
        author_dev_links["predictive_model_confidence"] < 0.97
    ].iterrows():
        # If the relationship was removed, add it to the set
        relationships_removed_from_confidence_thresh.add(
            (relationship_row["researcher_id"], relationship_row["developer_account_id"])
        )

    # Get the set of dev_researcher_links for the devs
    dev_researcher_links = researcher_dev_links.loc[
        researcher_dev_links["developer_account_id"].isin(dev_ids)
    ].sort_values("predictive_model_confidence", ascending=False)

    # Drop duplicates by dev_id (keeping first)
    # as we may have accidently matched the same author to the multiple devs
    dev_researcher_links = dev_researcher_links.drop_duplicates(
        subset=["researcher_id"],
        keep="first",
    )

    # Drop any dev researcher links that have less than 97% confidence
    dev_researcher_links_filtered = dev_researcher_links.loc[
        dev_researcher_links["predictive_model_confidence"] >= 0.97
    ]
    for _, relationship_row in dev_researcher_links.loc[
        dev_researcher_links["predictive_model_confidence"] < 0.97
    ].iterrows():
        # If the relationship was removed, add it to the set
        relationships_removed_from_confidence_thresh.add(
            (relationship_row["researcher_id"], relationship_row["developer_account_id"])
        )

    # Get the number of authors who were devs on this paper
    n_author_devs = 0
    n_author_non_devs = 0
    for author_id in author_ids:
        author_dev_ids = author_dev_links_filtered.loc[
            author_dev_links_filtered["researcher_id"] == author_id
        ]["developer_account_id"].unique()

        # Count
        if any(author_dev_id in dev_ids for author_dev_id in author_dev_ids):
            n_author_devs += 1
        else:
            n_author_non_devs += 1

    # Get the number of devs who aren't authors
    n_non_author_devs = 0
    for dev_id in dev_ids:
        dev_researcher_ids = dev_researcher_links_filtered.loc[
            dev_researcher_links_filtered["developer_account_id"] == dev_id
        ]["researcher_id"].unique()

        # No dev should be matched to more than 3 authors
        if len(dev_researcher_ids) > 3:
            continue

        # Count
        if not any(dev_researcher_id in author_ids for dev_researcher_id in dev_researcher_ids):
            n_non_author_devs += 1

    # Append
    team_composition_rows.append(
        {
            "document_id": row["document_id"],
            "repository_id": row["repository_id"],
            "author_ids": author_ids.tolist(),
            "dev_ids": dev_ids.tolist(),
            "n_authors": n_authors,
            "n_author_non_devs": n_author_non_devs,
            "n_author_devs": n_author_devs,
            "n_non_author_devs": n_non_author_devs,
            "n_devs": n_devs,
        }
    )

# Create dataframe
team_composition = pd.DataFrame(team_composition_rows)

# Merge with docs_w_1_citation
team_composition = team_composition.merge(
    docs_w_1_citation,
    left_on=["document_id", "repository_id"],
    right_on=["document_id", "repository_id"],
)

# Filter out papers with less than 3 authors or 1 dev
team_composition = team_composition.loc[team_composition["n_authors"] >= 3]
team_composition = team_composition.loc[team_composition["n_devs"] >= 1]

# Convert datetimes to datetime
team_composition["publication_date"] = pd.to_datetime(
    team_composition["publication_date"],
    utc=True,
)
team_composition["last_pushed_datetime"] = pd.to_datetime(
    team_composition["last_pushed_datetime"],
    utc=True,
)
team_composition["creation_datetime"] = pd.to_datetime(
    team_composition["creation_datetime"],
    utc=True,
)

# Calculate years since publication from 2024-11-01
team_composition["years_since_publication"] = (
    pd.to_datetime("2024-11-01", utc=True) - team_composition["publication_date"]
).dt.days / 365.25

# Calculate repo creation to last push
team_composition["repo_commit_duration"] = (
    team_composition["last_pushed_datetime"] - team_composition["creation_datetime"]
).dt.days / 365.25

# Create a "days_since_last_push" column
team_composition["days_from_publication_to_last_push"] = (
    team_composition["last_pushed_datetime"] - team_composition["publication_date"]
).dt.days

# Create a "days_since_last_push" column
team_composition["days_from_publication_to_last_push"] = (
    team_composition["last_pushed_datetime"] - team_composition["publication_date"]
).dt.days

# Must have a push within 90 days of publication
team_comp_no_push_after_pub = team_composition.loc[
    team_composition["days_from_publication_to_last_push"] <= 90
].copy()

# Create a "publication_year" column
team_comp_no_push_after_pub["publication_year"] = team_comp_no_push_after_pub[
    "publication_date"
].dt.year

# Drop columns that would conflict with conversion to float
team_comp_no_push_after_pub = team_comp_no_push_after_pub.drop(
    columns=[
        "publication_date",
        "last_pushed_datetime",
        "creation_datetime",
    ]
)

# Get the number of authors which would be the 97th percentile
n_authors_97th_percentile = team_comp_no_push_after_pub["n_authors"].quantile(0.97)

# Remove rows that are greater than 97th percentile for total authors
team_comp_no_push_after_pub = team_comp_no_push_after_pub.loc[
    team_comp_no_push_after_pub["n_authors"]
    <= team_comp_no_push_after_pub["n_authors"].quantile(0.97)
]

# Get columns needed for counts table
team_comp_no_push_after_pub_counts_table = team_comp_no_push_after_pub[
    [
        "document_id",
        "repository_id",
        "author_ids",
        "dev_ids",
        "article_type",
        "domain",
        "is_open_access",
    ]
].copy()

# Drop author_ids and dev_ids from original dataframe
team_comp_no_push_after_pub = team_comp_no_push_after_pub.drop(
    columns=["author_ids", "dev_ids"]
)

# Add dummies
team_comp_no_push_after_pub_dummies = pd.get_dummies(
    team_comp_no_push_after_pub,
    columns=["article_type", "domain"],
    drop_first=True,
)

# Cast all to float
team_comp_no_push_after_pub_dummies = team_comp_no_push_after_pub_dummies.astype(float)

# Create metrics for overall
metric_str_fmt = "{met_mean:.1f} ± {met_std:.1f}"
total_auths = metric_str_fmt.format(
    met_mean=team_comp_no_push_after_pub["n_authors"].mean(),
    met_std=team_comp_no_push_after_pub["n_authors"].std(),
)
total_auth_devs = metric_str_fmt.format(
    met_mean=team_comp_no_push_after_pub["n_author_devs"].mean(),
    met_std=team_comp_no_push_after_pub["n_author_devs"].std(),
)
total_auth_non_devs = metric_str_fmt.format(
    met_mean=team_comp_no_push_after_pub["n_author_non_devs"].mean(),
    met_std=team_comp_no_push_after_pub["n_author_non_devs"].std(),
)
total_non_auth_devs = metric_str_fmt.format(
    met_mean=team_comp_no_push_after_pub["n_non_author_devs"].mean(),
    met_std=team_comp_no_push_after_pub["n_non_author_devs"].std(),
)

# Get the number of papers which have great than zero non_author_devs
n_papers_w_non_auth_devs = len(team_comp_no_push_after_pub.loc[
    team_comp_no_push_after_pub["n_non_author_devs"] > 0
])
pct_papers_w_non_auth_devs = round(
    (n_papers_w_non_auth_devs / len(team_comp_no_push_after_pub)) * 100, 1
)
median_non_author_devs_for_papers_w_non_auth_devs = team_comp_no_push_after_pub.loc[
    team_comp_no_push_after_pub["n_non_author_devs"] > 0
]["n_non_author_devs"].median()
mean_non_author_devs_for_papers_w_non_auth_devs = metric_str_fmt.format(
    met_mean=team_comp_no_push_after_pub.loc[
        team_comp_no_push_after_pub["n_non_author_devs"] > 0
    ]["n_non_author_devs"].mean(),
    met_std=team_comp_no_push_after_pub.loc[
        team_comp_no_push_after_pub["n_non_author_devs"] > 0
    ]["n_non_author_devs"].std(),
)

# Merge counts table with dataset sources
team_comp_no_push_after_pub_counts_table = team_comp_no_push_after_pub_counts_table.merge(
    doc_repo_links[[
        "document_id",
        "repository_id",
        "dataset_source_id",
    ]],
    on=["document_id", "repository_id"],
    how="left",
).merge(
    dataset_sources.rename(
        columns={
            "id": "dataset_source_id", "name": "dataset_source_name"
        }
    ),
    on="dataset_source_id",
    how="left",
)

# Get counts by:
# - domain
# - article type
# - open access status
# - dataset source
# - total

def _get_counts_by_column(
    df: pd.DataFrame,
    column_name: str,
    category: str,
    subset_replace: dict | None = None,
) -> dict:
    counts = []
    for value in df[column_name].unique():
        subset = df.loc[df[column_name] == value]

        # Replace value if subset_replace is provided
        if subset_replace is not None and value in subset_replace:
            value = subset_replace[value]

        # Get counts
        author_ids = set(subset["author_ids"].explode().dropna().unique())
        dev_ids = set(subset["dev_ids"].explode().dropna().unique())

        # Append counts
        counts.append({
            category: value,
            "n_article_repo_pairs": len(subset),
            "n_authors": len(author_ids),
            "n_devs": len(dev_ids),
        })

    return counts

team_comp_no_push_after_pub_counts_domain = _get_counts_by_column(
    team_comp_no_push_after_pub_counts_table,
    "domain",
    "domain",
)
team_comp_no_push_after_pub_counts_article_type = _get_counts_by_column(
    team_comp_no_push_after_pub_counts_table,
    "article_type",
    "doc_type",
)
team_comp_no_push_after_pub_counts_open_access = _get_counts_by_column(
    team_comp_no_push_after_pub_counts_table,
    "is_open_access",
    "access_status",
    subset_replace={
        0: "Closed",
        1: "Open",
    },
)
team_comp_no_push_after_pub_counts_dataset_source = _get_counts_by_column(
    team_comp_no_push_after_pub_counts_table,
    "dataset_source_name",
    "data_source",
)

team_comp_no_push_after_pub_total_pairs = len(
    team_comp_no_push_after_pub_counts_table
)
team_comp_no_push_after_pub_total_authors = team_comp_no_push_after_pub_counts_table[
    "author_ids"
].explode().dropna().nunique()
team_comp_no_push_after_pub_total_devs = team_comp_no_push_after_pub_counts_table[
    "dev_ids"
].explode().dropna().nunique()

Understanding the composition and dynamics of software development teams provides essential context for analyzing how code contributions relate to scientific recognition and impact. To ensure reliable analysis, we focus on a subset of our article-repository pairs that meet several filtering conditions. First, we require that each article-repository pair have at least one citation, which helps ensure the research has received a basic level of engagement from the scientific community. Next, we require that repository commit activity must stop prior to 90 days past the date of article publication. Disallowing long-term projects ensures we do not include projects that may add additional code contributors later while still allowing a grace period during which developers can update repositories with additional documentation and publication information. We then subset the data to only include article-repository pairs with research teams of typical size by removing those with fewer than three authors and more than `{python} int(n_authors_97th_percentile)` authors, the 97th percentile for research team size. Finally, we filter out any author-developer pairs associated with these projects with predictive model confidence of less than 0.97 to ensure that we only include high-confidence matches[^3]. This filtering process results in a dataset of `{python} len(team_comp_no_push_after_pub)` article-repository pairs. A table with the counts of article-repository pairs, authors, and developers by data sources, domains, document types, and access status for this filtered dataset is shown in @tbl-team-comp-no-push-after-pub-counts.

[^3]: @fig-dist-of-author-dev-pred-confidence shows the distribution of predictive model confidence scores for author-developer pairs to justify this threshold. We chose the 0.97 threshold to ensure that we only include high-confidence matches while retaining a large proportion of the data (\~90,000 author-developer pairs) as less than 3000 author-developer-account pairs have a confidence less than 0.97 in the whole unfiltered dataset.

Within this filtered dataset, we categorized individuals into three groups: code-contributing authors (CC-A) who both authored papers and contributed code to associated repositories, non-code-contributing authors (NCC-A) who authored papers but showed no evidence of code contributions, and code-contributing non-authors (CC-NA) who contributed code but received no authorship recognition. This categorization revealed that papers in our dataset typically have `{python} total_auths` total authors, with `{python} total_auth_devs` code-contributing authors and `{python} total_auth_non_devs` non-code-contributing authors. Beyond the author list, papers averaged `{python} total_non_auth_devs` code-contributing non-authors. @tbl-team-composition-counts details these distributions by domain, article type, and open access status.

Perhaps most striking is our finding that `{python} n_papers_w_non_auth_devs` papers (`{python} pct_papers_w_non_auth_devs`%) have at least one code contributor who did not receive authorship recognition. Within this substantial subset of papers, we found an average of `{python} mean_non_author_devs_for_papers_w_non_auth_devs` unrecognized code contributors per paper. On average, only one code-contributing author per paper aligns with previous research by @Larivire2020InvestigatingTD, showing that technical tasks like data curation, formal analysis, visualization, and software development typically fall to first authors. However, our finding that over a quarter of papers have unrecognized code contributors suggests a more complex dynamic between software development and authorship recognition.


In [ ]:
# Create a copy of the team_comp_no_push_after_pub dataframe
team_comp_no_push_after_pub_counts = team_comp_no_push_after_pub.copy()

# Replace binary values in "is_open_access" to "Open Access" and "Closed Access"
team_comp_no_push_after_pub_counts["is_open_access"] = (
    team_comp_no_push_after_pub_counts.apply(
        lambda x: "Open" if x["is_open_access"] == 1 else "Closed",
        axis=1,
    )
)

# Count team composition values and always for each control variable
control_vars = {
    "is_open_access": "OA Status",
    "domain": "Domain",
    "article_type": "Article Type",
    "Overall": "Overall",
}
control_var_tables = {}
for control_var, control_display_name in control_vars.items():
    if control_var == "Overall":
        mean_table = (
            team_comp_no_push_after_pub_counts[
                ["n_authors", "n_author_non_devs", "n_author_devs", "n_non_author_devs"]
            ]
            .mean()
            .to_frame()
            .T
        )
        std_table = (
            team_comp_no_push_after_pub_counts[
                ["n_authors", "n_author_non_devs", "n_author_devs", "n_non_author_devs"]
            ]
            .std()
            .to_frame()
            .T
        )

        # Merge and format to string
        count_table_rows = []
        for _, row in mean_table.iterrows():
            std_row = std_table.loc[std_table.index == row.name]
            count_table_rows.append(
                {
                    "Overall": "",
                    "n_authors": f"{row['n_authors']:.1f} ± {std_row['n_authors'].iloc[0]:.1f}",
                    "n_author_non_devs": f"{row['n_author_non_devs']:.1f} ± {std_row['n_author_non_devs'].iloc[0]:.1f}",
                    "n_author_devs": f"{row['n_author_devs']:.1f} ± {std_row['n_author_devs'].iloc[0]:.1f}",
                    "n_non_author_devs": f"{row['n_non_author_devs']:.1f} ± {std_row['n_non_author_devs'].iloc[0]:.1f}",
                }
            )

    else:
        # Instead of taking median, let's return the mean and std in the following format
        # "mean ± std"
        mean_table = (
            team_comp_no_push_after_pub_counts.groupby(control_var)[
                ["n_authors", "n_author_non_devs", "n_author_devs", "n_non_author_devs"]
            ]
            .mean()
            .reset_index()
        )
        std_table = (
            team_comp_no_push_after_pub_counts.groupby(control_var)[
                ["n_authors", "n_author_non_devs", "n_author_devs", "n_non_author_devs"]
            ]
            .std()
            .reset_index()
        )

        # Merge and format to string
        count_table_rows = []
        for _, row in mean_table.iterrows():
            std_row = std_table.loc[std_table[control_var] == row[control_var]]
            count_table_rows.append(
                {
                    control_var: row[control_var],
                    "n_authors": f"{row['n_authors']:.1f} ± {std_row['n_authors'].iloc[0]:.1f}",
                    "n_author_non_devs": f"{row['n_author_non_devs']:.1f} ± {std_row['n_author_non_devs'].iloc[0]:.1f}",
                    "n_author_devs": f"{row['n_author_devs']:.1f} ± {std_row['n_author_devs'].iloc[0]:.1f}",
                    "n_non_author_devs": f"{row['n_non_author_devs']:.1f} ± {std_row['n_non_author_devs'].iloc[0]:.1f}",
                }
            )

    # Create dataframe
    count_table = pd.DataFrame(count_table_rows)

    # Change name of the control_var column to "Subset"
    count_table = count_table.rename(columns={control_var: "Subset"})

    # Order columns
    count_table = count_table[
        ["Subset", "n_authors", "n_author_non_devs", "n_author_devs", "n_non_author_devs"]
    ]
    
    # Rename columns
    count_table = count_table.rename(
        columns={
            "n_authors": "Total Authors",
            "n_author_non_devs": "NCC-A",
            "n_author_devs": "CC-A",
            "n_non_author_devs": "CC-NA",
        }
    )

    # Order alphabetically
    count_table = count_table.sort_values("Subset")
    
    # Append
    control_var_tables[control_display_name] = count_table

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-team-composition-counts
#| tbl-cap: Mean and Standard Deviation of Non-Code-Contributing Authors (NCC-A), Code-Contributing Authors (CC-A), and Code-Contributing Non-Authors (CC-NA) Research Team Members by Domain, Article Type, and Open Access Status. Only includes research teams from article-repository pairs with a most recent commit no later than 90 days after publication and excludes research teams in the top 3% of total author sizes.

# Construct multi-row span HTML table
# Columns should be: "Authors", "Code Contributing Authors", "Non-Author Code Contributors"
# Rows should be:
# "Open Access Status", "Domain", "Article Type"

# HTML templates
count_piece_inital_row_template = """
<tr>
  <td rowspan="{n_rows}">{row_name}</td>
  <td>{value_name}</td>
  <td>{n_authors}</td>
  <td>{n_author_non_devs}</td>
  <td>{n_author_devs}</td>
  <td>{n_non_author_devs}</td>
</tr>
""".strip()

count_piece_bolded_row_template = """
<tr>
  <td rowspan="{n_rows}"><b>{row_name}</b></td>
  <td><b>{value_name}</b></td>
  <td><b>{n_authors}</b></td>
  <td><b>{n_author_non_devs}</b></td>
  <td><b>{n_author_devs}</b></td>
  <td><b>{n_non_author_devs}</b></td>
</tr>
""".strip()

count_piece_subsequent_row_template = """
<tr>
  <td>{value_name}</td>
  <td>{n_authors}</td>
  <td>{n_author_non_devs}</td>
  <td>{n_author_devs}</td>
  <td>{n_non_author_devs}</td>
</tr>
""".strip()

# Iter over stats portions (and total)
count_portions_html = []
for key, count_table in control_var_tables.items():
    count_portion_html = []
    for i, control_value in enumerate(count_table["Subset"].unique()):
        if i == 0:
            if control_value == "":  # empty control value is for the "Overall" table
                count_portion_html.append(
                    count_piece_bolded_row_template.format(
                        n_rows=len(count_table),
                        row_name=key,
                        value_name=control_value,
                        n_authors=count_table.loc[
                            count_table["Subset"] == control_value,
                            "Total Authors",
                        ].iloc[0],
                        n_author_non_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "NCC-A",
                        ].iloc[0],
                        n_author_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "CC-A",
                        ].iloc[0],
                        n_non_author_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "CC-NA",
                        ].iloc[0],
                    )
                )
            else:
                count_portion_html.append(
                    count_piece_inital_row_template.format(
                        n_rows=len(count_table),
                        row_name=key,
                        value_name=control_value,
                        n_authors=count_table.loc[
                            count_table["Subset"] == control_value,
                            "Total Authors",
                        ].iloc[0],
                        n_author_non_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "NCC-A",
                        ].iloc[0],
                        n_author_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "CC-A",
                        ].iloc[0],
                        n_non_author_devs=count_table.loc[
                            count_table["Subset"] == control_value,
                            "CC-NA",
                        ].iloc[0],
                    )
                )
        else:
            count_portion_html.append(
                count_piece_subsequent_row_template.format(
                    value_name=control_value,
                    n_authors=count_table.loc[
                        count_table["Subset"] == control_value,
                        "Total Authors",
                    ].iloc[0],
                    n_author_non_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "NCC-A",
                    ].iloc[0],
                    n_author_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "CC-A",
                    ].iloc[0],
                    n_non_author_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "CC-NA",
                    ].iloc[0],
                )
            )

    count_portions_html.append("\n".join(count_portion_html))

# Concat and wrap in table
count_table_html = f"""
<table>
  <tr>
    <th><b>Control</b></th>
    <th><b>Subset</b></th>
    <th><b>Total Authors</b></th>
    <th><b>NCC-A</b></th>
    <th><b>CC-A</b></th>
    <th><b>CC-NA</b></th>
  </tr>
  {" ".join(count_portions_html)}
</table>
""".strip()

IPython.display.HTML(count_table_html)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
# LaTeX templates
count_piece_initial_row_template = """    \\multirow{{{n_rows}}}{{*}}{{\\textbf{{{row_name}}}}} & \\cellcolor{{gray!10}}{value_name} & \\cellcolor{{gray!10}}{n_authors} & \\cellcolor{{gray!10}}{n_author_non_devs} & \\cellcolor{{gray!10}}{n_author_devs} & \\cellcolor{{gray!10}}{n_non_author_devs} \\\\"""

count_piece_bolded_row_template = """    \\multirow{{{n_rows}}}{{*}}{{\\textbf{{{row_name}}}}} & \\textbf{{{value_name}}} & \\textbf{{{n_authors}}} & \\textbf{{{n_author_non_devs}}} & \\textbf{{{n_author_devs}}} & \\textbf{{{n_non_author_devs}}} \\\\"""

count_piece_subsequent_row_template = """    & {color_cmd}{value_name} & {color_cmd}{n_authors} & {color_cmd}{n_author_non_devs} & {color_cmd}{n_author_devs} & {color_cmd}{n_non_author_devs} \\\\"""

# Table header and footer templates
table_header = """\\begin{table}
\\centering
\\small
\\caption{Mean and Standard Deviation of Non-Code-Contributing Authors (NCC-A), Code-Contributing Authors (CC-A), and Code-Contributing Non-Authors (CC-NA) Research Team Members by Domain, Article Type, and Open Access Status. Only includes research teams from article-repository pairs with a most recent commit no later than 90 days after publication and excludes research teams in the top $3\%$ of total author sizes.}
\\label{tbl-team-composition-counts}
\\begin{tabular}{llrrrr}
\\toprule\\textbf{Control} & \\textbf{Subset} & \\textbf{Total Authors} & \\textbf{NCC-A} & \\textbf{CC-A} & \\textbf{CC-NA} \\\\
\\midrule"""

table_footer = """\\bottomrule
\\end{tabular}
\\end{table}"""

# Generate table content
count_portions_latex = []
for key, count_table in control_var_tables.items():
    count_portion_latex = []
    
    # Get unique control values
    control_values = count_table["Subset"].unique()

    # Handle overall
    if key == "Overall":
        count_portion_latex.append(
            count_piece_bolded_row_template.format(
                n_rows=1,
                row_name=key,
                value_name="",
                n_authors=total_auths,
                n_author_non_devs=total_auth_non_devs,
                n_author_devs=total_auth_devs,
                n_non_author_devs=total_non_auth_devs,
            ).rstrip()
        )
    
    else:
        # First row with category label
        first_value = control_values[0]
        count_portion_latex.append(
            count_piece_initial_row_template.format(
                n_rows=len(control_values),
                row_name=key,
                value_name=first_value,
                n_authors=count_table.loc[
                    count_table["Subset"] == first_value,
                    "Total Authors",
                ].iloc[0],
                n_author_non_devs=count_table.loc[
                    count_table["Subset"] == first_value,
                    "NCC-A",
                ].iloc[0],
                n_author_devs=count_table.loc[
                    count_table["Subset"] == first_value,
                    "CC-A",
                ].iloc[0],
                n_non_author_devs=count_table.loc[
                    count_table["Subset"] == first_value,
                    "CC-NA",
                ].iloc[0],
            ).rstrip()
        )
        
        # Subsequent rows with alternating colors
        for i, control_value in enumerate(control_values[1:]):
            color_cmd = "\\cellcolor{gray!10}" if i % 2 == 1 else ""
            count_portion_latex.append(
                count_piece_subsequent_row_template.format(
                    color_cmd=color_cmd,
                    value_name=control_value,
                    n_authors=count_table.loc[
                        count_table["Subset"] == control_value,
                        "Total Authors",
                    ].iloc[0],
                    n_author_non_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "NCC-A",
                    ].iloc[0],
                    n_author_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "CC-A",
                    ].iloc[0],
                    n_non_author_devs=count_table.loc[
                        count_table["Subset"] == control_value,
                        "CC-NA",
                    ].iloc[0],
                ).rstrip()
            )

    section_latex = "\n".join(count_portion_latex)
    
    if key != list(control_var_tables.keys())[-1]:  # Don't add midrule after last section
        section_latex += "\\midrule"
    count_portions_latex.append(section_latex)

# Combine all parts
count_table_latex = f"""{table_header}
{"\n".join(count_portions_latex)}
{table_footer}"""

IPython.display.Latex(count_table_latex)

:::

When examining these patterns over time and across different team sizes (@fig-contributor-type-by-time-and-size), we found that the number of code-contributing authors and unrecognized contributors has remained relatively stable. This stability over time suggests that while the exclusion of code contributors from authorship is not worsening, it represents a persistent feature of scientific software development rather than a historical artifact or transition period in research practices. Similarly, the number of code-contributing non-authors remains constant even as team size grows, indicating that larger research teams do not necessarily adopt more inclusive authorship practices for code contributors, despite representing broader collaborative efforts.


In [ ]:
#| label: fig-contributor-type-by-time-and-size
#| fig-cap: Average number of contributors per article, by contribution type along with A) the year the article was published, and B) the total number of authors included on the article. Only includes research teams from article-repository pairs with a most recent commit no later than 90 days after publication and excludes research teams in the top 3% of total author sizes for publication years with 50 or more articles. Shaded areas show the 95% confidence interval for the mean.

# Make a copy for team size comparison
team_comp_by_year_and_size = team_comp_no_push_after_pub.copy()

# Rename columns
team_comp_by_year_and_size = team_comp_by_year_and_size.rename(
    columns={
        "n_authors": "Total Authors",
        "publication_year": "Publication Year",
        "n_author_devs": "Code-Contrib. Authors",
        "n_author_non_devs": "Non-Code-Contrib. Authors",
        "n_non_author_devs": "Code-Contrib. Non-Authors",
    }
)

# Get value count of publication year
year_counts = team_comp_no_push_after_pub["publication_year"].value_counts()

# Drop years with less than 50 publications
team_comp_by_year_and_size = team_comp_by_year_and_size.loc[
    team_comp_no_push_after_pub["publication_year"].isin(
        year_counts.loc[lambda x: x >= 50].index
    )
]

# First separate out the year and authors
team_comp_by_year_and_size = team_comp_by_year_and_size.melt(
    id_vars=[
        "document_id",
        "Non-Code-Contrib. Authors",
        "Code-Contrib. Authors",
        "Code-Contrib. Non-Authors",
    ],
    value_vars=[
        "Publication Year",
        "Total Authors",
    ],
    var_name="comparison_var",
    value_name="value",
)

# Melt again to get the contributor type
team_comp_by_year_and_size = team_comp_by_year_and_size.melt(
    id_vars=[
        "document_id",
        "comparison_var",
        "value",
    ],
    value_vars=[
        "Non-Code-Contrib. Authors",
        "Code-Contrib. Authors",
        "Code-Contrib. Non-Authors",
    ],
    var_name="Contributor Type",
    value_name="Count",
)

# Plot
g = sns.relplot(
    data=team_comp_by_year_and_size,
    x="value",
    y="Count",
    col="comparison_var",
    hue="Contributor Type",
    hue_order=[
        "Non-Code-Contrib. Authors",
        "Code-Contrib. Authors",
        "Code-Contrib. Non-Authors",
    ],
    kind="line",
    estimator="mean",
    legend=True,
    facet_kws={"sharex": False, "sharey": False},
    height=4,
)

# Iter over axes and set x labels
for i, ax in enumerate(g.axes.flat):
    if i == 0:
        ax.set_title("A")
        ax.set_xlabel("Publication Year")
    else:
        ax.set_title("B")
        ax.set_xlabel("Total Authors")

# Move legend to below the plot
sns.move_legend(
    g,
    bbox_to_anchor=(0.4, -0.0001),
    loc="upper center",
    ncol=3,
    fontsize=12,
    title_fontsize=14,
)

### Modeling Article Citations

Building upon previous work examining the effects of team size and team diversity on scientific impact and software quality (see @sec-background), we investigate how the number of code contributors within a research team may be associated with an article's research impact. We hypothesized that more code contributors might signal greater technical complexity in research, which may be associated with higher citation counts as the community builds upon more technically sophisticated works.

Using our filtered dataset of article-repository pairs (@tbl-team-comp-no-push-after-pub-counts), we conducted multiple regression analyses to examine these relationships while controlling for various factors. Without controlling for domain, open access, or article type differences (@tbl-article-composition-overall), our analysis revealed a modest positive association between the number of code contributing authors and article citations, with each code-contributing author associated with a 5.1% increase in article citations (p \< 0.001).

When controlling for article type (@tbl-article-composition-type), we observed divergent patterns between preprints and research articles. For preprints, each code-contributing non-author was associated with a statistically significant 3.2% decrease in citations (p \< 0.005). In contrast, research articles showed more positive associations: we found a significant positive relationship between code-contributing authors and citations (p \< 0.001), though we cannot estimate the precise magnitude due to the non-significant main effect in the model. Additionally, each code-contributing non-author was associated with a 0.1% increase in expected citations for research articles (p \< 0.001).

Based on these findings, we ***partially accept*** our hypothesis (*H1*) that "the number of individuals contributing code to a publication's associated repository positively correlates with the article's citation count." Several important nuances qualify this acceptance: the relationship is statistically significant but modest in magnitude and differs substantially between research articles (positive association) and preprints (negative association for non-author code contributors). These variations suggest that the relationship between code contributions and citation impact is context-dependent and more complex than initially hypothesized.


In [ ]:
def compute_article_level_models(
    y_col: str,
    data: pd.DataFrame,
    glm_family: sm.families.Family,
) -> dict[str, sm.GLM]:
    # Remove outliers
    no_outliers = data[
        data[y_col].between(
            data[y_col].quantile(0.03),
            data[y_col].quantile(0.97),
        )
    ].copy()

    # Remove nans
    no_outliers = no_outliers.dropna(subset=[y_col])

    # Replace names
    no_outliers = no_outliers.rename(
        columns={
            "n_authors": "Total Authors",
            # "n_author_non_devs": "Non-Code-Contrib. Authors",
            "n_author_devs": "Code-Contrib. Authors",
            "n_non_author_devs": "Code-Contrib. Non-Authors",
        }
    )

    # Replace other names (except const.) by removing `_` and title casing
    no_outliers = no_outliers.rename(
        columns=lambda x: x.replace("_", " ").title() if x not in ["const", y_col] else x
    )

    # Common features to use in all models
    required_features = [
        y_col,
        "Total Authors",
        # "n_author_non_devs",
        "Code-Contrib. Authors",
        "Code-Contrib. Non-Authors",
        "Years Since Publication",
    ]

    

    # Iter over different control variables and create models for each
    models = {}
    for control_var in [
        "no-control",
        "Article Type",
        "Domain",
        "Is Open Access",
    ]:
        if control_var != "no-control":
            # Get control variable list
            control_variables = [
                col for col in no_outliers.columns if col.startswith(control_var)
            ]

            # Create control variable subset of the data
            control_var_subset = no_outliers[required_features + control_variables].copy()

            # Create interactions
            for coding_status_col in ["Code-Contrib. Authors", "Code-Contrib. Non-Authors"]:
                for control_col in control_variables:
                    control_var_subset[f"{coding_status_col} × {control_col}"] = (
                        control_var_subset[coding_status_col]
                        * control_var_subset[control_col]
                    )
        else:
            control_var_subset = no_outliers[required_features].copy()

        # Drop inf and nan
        control_var_subset = control_var_subset.replace(
            [float("inf"), -float("inf")], float("nan")
        ).dropna()

        # Create x and y
        y = control_var_subset[y_col]
        x = control_var_subset.drop(columns=[y_col])
        x = sm.add_constant(x)

        # Fit model
        model = sm.GLM(y, x, family=glm_family).fit()
        models[control_var] = model

    return models

In [ ]:
# LaTeX templates
model_results_row_template = """    {bold_str}\\cellcolor{{{color}}}{var_name}{stars} & {bold_str}\\cellcolor{{{color}}}{coef:.2f} & {bold_str}\\cellcolor{{{color}}}{p:.2f} & {bold_str}\\cellcolor{{{color}}}{ci_low:.2f} & {bold_str}\\cellcolor{{{color}}}{ci_high:.2f} \\\\"""

# Table header and footer templates
model_results_table_header = """\\begin{tabular}{l*{6}{r}}
\\toprule
\\textbf{Variable} & \\textbf{coef} & \\textbf{P>|z|} & \\multicolumn{2}{c}{\\textbf{[0.025 0.975]}} \\\\
\\midrule"""

model_results_table_footer = """\\bottomrule
\\end{tabular}"""

def convert_model_results_to_printable_pdf_ready(
    model: sm.GLM,
    tbl_cap: str,
    tbl_label: str,
) -> tuple[str, pd.DataFrame]:
    # Get only the dataframe
    summary_simple_tab = model.summary().tables[1]
    model_results_df = pd.read_html(StringIO(summary_simple_tab.as_html()), header=0, index_col=0)[0]

    # Add exponentiated coef
    model_results_df["exp(coef)"] = np.exp(model_results_df["coef"])

    # Keep only the specified columns
    model_results_df = model_results_df[
        ["exp(coef)", "coef", "z", "P>|z|", "[0.025", "0.975]"]
    ]

    # Set index name to "variable"
    model_results_df.index.name = "variable"
    model_results_df = model_results_df.reset_index()

    rows_latex = []
    for i, result_row in model_results_df.iterrows():
        # Determine if row should be bold and if stars should be added
        is_significant = result_row["P>|z|"] < 0.05
        stars = ""
        if result_row["P>|z|"] < 0.001:
            stars = "~$^{***}$"
        elif result_row["P>|z|"] < 0.01:
            stars = "~$^{**}$"
        elif result_row["P>|z|"] < 0.05:
            stars = "~$^{*}$"
        
        
        
        # Handle bolding - only for significant rows
        if is_significant:
            bold_str = "\\bfseries"
        else:
            bold_str = ""
        
        # Set background color for alternating rows
        color = "gray!10" if i % 2 == 1 else "white"
        
        # Format the row
        rows_latex.append(
            model_results_row_template.format(
                bold_str=bold_str,
                var_name=result_row["variable"].replace("_", " "),
                # exp_coef=result_row["exp(coef)"],
                coef=result_row["coef"],
                # z=result_row["z"],
                p=result_row["P>|z|"],
                ci_low=result_row["[0.025"],
                ci_high=result_row["0.975]"],
                stars=stars,
                color=color
            )
        )

    # Combine all parts
    regression_table_latex = f"""\\begin{{table}}
\\centering
\\caption{{{tbl_cap}}}
\\label{{{tbl_label}}}
\\label{{tbl-reg-results}}
{model_results_table_header}
{chr(10).join(rows_latex)}
{model_results_table_footer}
\\end{{table}}"""

    return regression_table_latex, model_results_df

In [ ]:
# Create models for cited_by_count
article_cited_by_count_models = compute_article_level_models(
    "cited_by_count",
    team_comp_no_push_after_pub_dummies,
    glm_family=sm.families.NegativeBinomial(),
)

## Characteristics of Scientific Code Contributors

### Author Positions of Code Contributing Authors


In [ ]:
def _run_posthocs(
    data: pd.DataFrame,
    split_var: str,
) -> list[dict[str, str]]:
    # Conduct pairwise posthocs
    binom_results = []
    for split_val in data[split_var].unique():
        split_val_subset = data.loc[
            data[split_var] == split_val
        ]

        # Run test
        results = binomtest(
            sum(split_val_subset["is_code_contributor"]),
            len(split_val_subset),
            0.5,
        )
        
        # Conduct binomial test
        binom_results.append(
            {
                split_var: split_val,
                "p": results.pvalue,
                "statistic": results.statistic,
                "n": len(split_val_subset),
                "n_code_contributors": sum(split_val_subset["is_code_contributor"]),
            }
        )

        # P adjust
        _, p_values, _, _ = multipletests(
            [result["p"] for result in binom_results],
            method="bonferroni",
        )

        # Replace p values
        for i, result in enumerate(binom_results):
            result["p"] = p_values[i]
        
    # Print results
    formatted_binom_results = []
    for result in binom_results:
        formatted_binom_results.append({
            split_var.replace("_", " ").title(): result[split_var].title(),
            "Coding": result["n_code_contributors"],
            "Total": result["n"],
            "p": result["p"],
        })

    return formatted_binom_results
    

def _run_chi2_and_posthocs(
    data: pd.DataFrame,
    control_var: str | None,
    split_var: str,
) -> list[dict[str, str | float | int]]:
    chi2_result_str_template = "χ2={stat:.1f}, p={p:.3f}, n={n}"
    
    return_vals = []

    if control_var is None:
        xtabs = pd.crosstab(
            data[split_var],
            data["is_code_contributor"],
        )
        chi2_result = chi2_contingency(xtabs)
        if chi2_result.pvalue < 0.05:
            posthoc_results = _run_posthocs(
                data,
                split_var=split_var,
            )

            for posthoc_result in posthoc_results:
                return_vals.append({
                    "Control": "Overall",
                    "Subset": "Overall",
                    **posthoc_result,
                })

        else:
            pass
    else:
        for control_val in data[control_var].unique():
            # Get data with control_val subset
            control_val_subset = data.loc[
                data[control_var] == control_val
            ]
            xtabs = pd.crosstab(
                control_val_subset[split_var],
                control_val_subset["is_code_contributor"],
            )
            chi2_result = chi2_contingency(xtabs)
            posthoc_results = _run_posthocs(
                control_val_subset,
                split_var=split_var,
            )

            for posthoc_result in posthoc_results:
                return_vals.append({
                    "Control": control_var.replace("_", " ").title(),
                    "Subset": control_val.replace("_", " ").title(),
                    **posthoc_result,
                })

    return return_vals

In [ ]:
doc_contribs_for_code_char = document_contributors.loc[
    document_contributors["document_id"].isin(team_comp_no_push_after_pub["document_id"])
].merge(
    team_comp_no_push_after_pub[[
        "document_id",
        "repository_id",
        "is_open_access",
        "domain",
        "article_type",
    ]]
)

# Iter over rows and check if the author is a dev on the same article-repo pair
doc_contribs_for_code_char_rows = []
for i, row in doc_contribs_for_code_char.iterrows():
    # Get devs
    this_repo_devs = repository_contributors.loc[
        repository_contributors["repository_id"] == row["repository_id"]
    ]

    # Get the set of researcher_dev_links for the authors
    author_dev_links = researcher_dev_links.loc[
        researcher_dev_links["researcher_id"] == row.researcher_id
    ].sort_values("predictive_model_confidence", ascending=False)

    # Drop duplicates by developer_account_id (keeping first)
    # as we may have accidently matched the same dev to the multiple authors
    author_dev_links = author_dev_links.drop_duplicates(
        subset=["developer_account_id"],
        keep="first",
    )

    # Drop any author dev links that have less than 97% confidence
    author_dev_links = author_dev_links.loc[
        author_dev_links["predictive_model_confidence"] >= 0.97
    ]

    # If no author dev links return same rows and "is_code_contributor" as False
    if len(author_dev_links) == 0:
        doc_contribs_for_code_char_rows.append(
            {
                **row.to_dict(),
                "is_code_contributor": False,
            }
        )

    # If there are more than three author dev links, ignore
    elif len(author_dev_links) > 3:
        continue
    
    else:
        # Add same rows and add "is_code_contributor" based on if developer_account_id is in this_repo_devs
        doc_contribs_for_code_char_rows.append(
            {
                **row.to_dict(),
                "is_code_contributor": any(
                    author_dev_link["developer_account_id"]
                    in this_repo_devs["developer_account_id"].unique()
                    for _, author_dev_link in author_dev_links.iterrows()
                ),
            }
        )

# Create dataframe
doc_contribs_for_code_char = pd.DataFrame(doc_contribs_for_code_char_rows)

# Replace binary values in "is_open_access" to "Open Access" and "Closed Access"
doc_contribs_for_code_char["is_open_access"] = (
    doc_contribs_for_code_char.apply(
        lambda x: "Open Access" if x["is_open_access"] == 1 else "Closed Access",
        axis=1,
    )
)

# Replace binary values in "is_corresponding" to "Corresponding" and "Not Corresponding"
doc_contribs_for_code_char["is_corresponding"] = (
    doc_contribs_for_code_char.apply(
        lambda x: "Corresponding" if x["is_corresponding"] == 1 else "Not Corresponding",
        axis=1,
    )
)

# Calculate the percent of first authors who are code contributors
first_author_code_contributors = doc_contribs_for_code_char.loc[
    doc_contribs_for_code_char["position"] == "first"
]["is_code_contributor"].sum()
percent_first_authors_code_contributors = (
    first_author_code_contributors
    / len(
        doc_contribs_for_code_char.loc[
            doc_contribs_for_code_char["position"] == "first"
        ]
    )
)
percent_first_authors_code_contributors_str = f"{percent_first_authors_code_contributors:.1%}"

middle_author_code_contributors = doc_contribs_for_code_char.loc[
    doc_contribs_for_code_char["position"] == "middle"
]["is_code_contributor"].sum()
percent_middle_authors_code_contributors = (
    middle_author_code_contributors
    / len(
        doc_contribs_for_code_char.loc[
            doc_contribs_for_code_char["position"] == "middle"
        ]
    )
)
percent_middle_authors_code_contributors_str = f"{percent_middle_authors_code_contributors:.1%}"

last_author_code_contributors = doc_contribs_for_code_char.loc[
    doc_contribs_for_code_char["position"] == "last"
]["is_code_contributor"].sum()
percent_last_authors_code_contributors = (
    last_author_code_contributors
    / len(
        doc_contribs_for_code_char.loc[
            doc_contribs_for_code_char["position"] == "last"
        ]
    )
)
percent_last_authors_code_contributors_str = f"{percent_last_authors_code_contributors:.1%}"

domain_code_char = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="domain",
        split_var="position",
    )
)
position_sorter = ["First", "Middle", "Last"]
domain_code_char["Position"] = pd.Categorical(
    domain_code_char["Position"],
    categories=position_sorter,
    ordered=True,
)
domain_code_char = domain_code_char.sort_values(["Subset", "Position"])

article_type_code_char = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="article_type",
        split_var="position",
    )
)
article_type_code_char["Position"] = pd.Categorical(
    article_type_code_char["Position"],
    categories=position_sorter,
    ordered=True,
)
article_type_code_char = article_type_code_char.sort_values(["Subset", "Position"])

open_access_status_code_char = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="is_open_access",
        split_var="position",
    )
)
open_access_status_code_char["Position"] = pd.Categorical(
    open_access_status_code_char["Position"],
    categories=position_sorter,
    ordered=True,
)
open_access_status_code_char = open_access_status_code_char.sort_values(["Subset", "Position"])

# Run chi2 and posthocs for overall author position
overall_code_char = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var=None,
        split_var="position",
    )
)
overall_code_char["Position"] = pd.Categorical(
    overall_code_char["Position"],
    categories=position_sorter,
    ordered=True,
)
overall_code_char = overall_code_char.sort_values(["Subset", "Position"])

Building upon previous work examining the relationship between authorship position and research contributions, we investigate how author position may relate to code contribution patterns. We hypothesized that first authors, traditionally contributing the bulk of intellectual and experimental work, are most likely to contribute code to a project. In contrast, middle and last authors often provide oversight and guidance and would be less likely to contribute code.

To analyze these patterns within our previously filtered dataset of article-repository pairs (@tbl-team-comp-no-push-after-pub-counts), we conducted Chi-square tests of independence between author position and code contribution status. These tests revealed significant associations between author position and likelihood of code contribution overall and when controlling for research domain, article type, and open access status (all p \< 0.01), indicating that the proportion of authors contributing code differs significantly based on author position. Following these significant associations, we examined the specific proportions across positions (@tbl-post-hoc-tests-on-author-positions): `{python} percent_first_authors_code_contributors_str` of first authors contributed code to their projects, compared to only `{python} percent_middle_authors_code_contributors_str` of middle authors and `{python} percent_last_authors_code_contributors_str` of last authors. The differences in these proportions remained statistically significant across all tested scenarios, regardless of research domain, article type, or open access status.

Based on these findings, we ***accept*** our hypothesis (*H2a*) that "first authors have higher code contribution rates than authors in other positions." The data demonstrates that the proportion of first authors who contribute code (`{python} percent_first_authors_code_contributors_str`) is significantly higher than the proportion of both middle authors (`{python} percent_middle_authors_code_contributors_str`) and last authors (`{python} percent_last_authors_code_contributors_str`). This relationship remains robust and statistically significant across all tested conditions, including variations in research domain, article type, and open access status, indicating a fundamental connection between authorship position and technical contribution in scientific research.

### Corresponding Status of Code Contributing Authors


In [ ]:
# Run chi2 and posthocs for overall corresponding status
domain_code_corresponding = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="domain",
        split_var="is_corresponding",
    )
).sort_values(["Subset", "Is Corresponding"])

article_type_code_corresponding = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="article_type",
        split_var="is_corresponding",
    )
).sort_values(["Subset", "Is Corresponding"])

open_access_status_code_corresponding = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var="is_open_access",
        split_var="is_corresponding",
    )
).sort_values(["Subset", "Is Corresponding"])

overall_code_corresponding = pd.DataFrame(
    _run_chi2_and_posthocs(
        doc_contribs_for_code_char,
        control_var=None,
        split_var="is_corresponding",
    )
).sort_values(["Subset", "Is Corresponding"])

# Calculate the percent of corresponding authors who are code contributors
corres_author_code_contributors = doc_contribs_for_code_char.loc[
    doc_contribs_for_code_char["is_corresponding"] == "Corresponding"
]["is_code_contributor"].sum()
percent_corresponding_authors_code_contributors = (
    corres_author_code_contributors
    / len(
        doc_contribs_for_code_char.loc[
            doc_contribs_for_code_char["is_corresponding"] == "Corresponding"
        ]
    )
)
percent_corresponding_authors_code_contributors_str = f"{percent_corresponding_authors_code_contributors:.1%}"

non_corres_author_code_contributors = doc_contribs_for_code_char.loc[
    doc_contribs_for_code_char["is_corresponding"] == "Not Corresponding"
]["is_code_contributor"].sum()
percent_non_corresponding_authors_code_contributors = (
    non_corres_author_code_contributors
    / len(
        doc_contribs_for_code_char.loc[
            doc_contribs_for_code_char["is_corresponding"] == "Not Corresponding"
        ]
    )
)
percent_non_corresponding_authors_code_contributors_str = f"{percent_non_corresponding_authors_code_contributors:.1%}"

Building upon our analysis of author position, we next examine how corresponding author status relates to code contribution patterns. We hypothesized that corresponding authors, who traditionally maintain research artifacts and serve as primary points of contact, would be more likely to contribute code compared to non-corresponding authors, as this role often involves responsibility for project resources and materials.

To analyze these relationships within our filtered dataset of article-repository pairs, we conducted Chi-square tests of independence between corresponding author status and code contribution status. Our analysis revealed patterns contrary to our initial hypothesis. The proportion of code contributors was low among both groups, with only `{python} percent_corresponding_authors_code_contributors_str` of corresponding authors and `{python} percent_non_corresponding_authors_code_contributors_str` of non-corresponding authors contributing code to their projects. Further examination (@tbl-post-hoc-tests-on-corresponding-status) showed that this pattern holds across nearly all conditions, with only a single exception: corresponding authors in closed-access publications showed no significant difference in their proportion of code contributors. However, this was tested with a sample of less than 200 authors.

Based on these findings, we ***reject*** our hypothesis (*H2b*) that "corresponding authors have higher code contribution rates than non-corresponding authors." Contrary to our expectations, our analysis revealed that the proportion of code contributors among corresponding authors (`{python} percent_corresponding_authors_code_contributors_str`) did not significantly differ from the proportion among non-corresponding authors (`{python} percent_non_corresponding_authors_code_contributors_str`). This pattern of similar proportions remained consistent across most studied conditions, with a single, small sample size exception in closed-access publications.

### Modeling Author H-Index


In [ ]:
# First, get the set of researchers who have at least 3 documents
researchers_w_3_docs = researchers.loc[
    researchers["id"].isin(
        document_contributors["researcher_id"]
        .value_counts()
        .loc[lambda x: x >= 3]
        .index
    )
]

# Use sample?
if USE_SAMPLE:
    researchers_w_3_docs = researchers_w_3_docs.sample(frac=0.02, random_state=12)

# Next, for each researcher, get the set of documents they have contributed to
researchers_w_3_docs = document_contributors.loc[
    document_contributors["researcher_id"].isin(researchers_w_3_docs["id"])
].merge(
    researchers_w_3_docs,
    left_on="researcher_id",
    right_on="id",
)

# Attach document for publication date
researchers_w_3_docs = researchers_w_3_docs.merge(
    documents[["id", "publication_date"]],
    left_on="document_id",
    right_on="id",
).drop(
    columns=["id"],
)

# Keep only certain columns
researchers_w_3_docs = researchers_w_3_docs[
    [
        "researcher_id",
        "document_id",
        "publication_date",
        "position",
        "is_corresponding",
        "works_count",
        "cited_by_count",
        "h_index",
        "i10_index",
        "two_year_mean_citedness",
    ]
]

# Next, for each researcher_doc, attach the document details (domain, reduced_doc_type)
researchers_w_3_docs = (
    researchers_w_3_docs.merge(
        document_topics[["document_id", "topic_id"]],
        left_on="document_id",
        right_on="document_id",
    )
    .merge(
        topics[["id", "domain_name"]],
        left_on="topic_id",
        right_on="id",
    )
    .drop(
        columns=["id", "topic_id"],
    )
    .merge(
        reduced_doc_types,
        left_on="document_id",
        right_on="document_id",
    )
)

# Now for each of these, we want to see if they have coded on the document
researchers_coded = []
individuals_removed_from_confidence_thresh = 0
for _, row in researchers_w_3_docs.iterrows():
    # Check for dev account
    dev_links = researcher_dev_links.loc[
        researcher_dev_links["researcher_id"] == row["researcher_id"]
    ]

    # Remove any links with less than 97% confidence
    before_drop_count = len(dev_links)
    dev_links = dev_links.loc[
        dev_links["predictive_model_confidence"] >= 0.97
    ]
    after_drop_count = len(dev_links)
    individuals_removed_from_confidence_thresh += (
        before_drop_count - after_drop_count
    )

    # Fast exit
    if len(dev_links) == 0:
        researchers_coded.append(
            {
                "researcher_id": row["researcher_id"],
                "document_id": row["document_id"],
                "coded_on_article": 0,
            }
        )
        continue

    # Skip this person if they have more than 3 links
    # Likely something went wrong
    if len(dev_links) > 3:
        continue

    # Get repos associated with document
    repo_links_for_doc = doc_repo_links.loc[
        doc_repo_links["document_id"] == row["document_id"]
    ]

    # Skip if there is more than 1 repo associated with the document
    # We just don't know how to handle these cases right now
    if len(repo_links_for_doc) > 1:
        continue

    # Also skip if 0
    if len(repo_links_for_doc) == 0:
        continue

    # Get the repo_id for the single repo
    repo_id = repo_links_for_doc["repository_id"].iloc[0]

    # Get the repo_contributors for this repository
    repo_contributors = repository_contributors.loc[
        repository_contributors["repository_id"] == repo_id
    ]

    # Check if any of the dev accounts are in the repo contribs
    researcher_coded = (
        len(
            set(repo_contributors["developer_account_id"].unique()).intersection(
                set(dev_links["developer_account_id"].unique()),
            )
        )
        > 0
    )

    # Finally assert any of the repo_contributors are the dev account
    # associated with the researcher
    researchers_coded.append(
        {
            "researcher_id": row["researcher_id"],
            "document_id": row["document_id"],
            "coded_on_article": int(researcher_coded),
        }
    )

# Create dataframe
researchers_coded_df = pd.DataFrame(researchers_coded)

# Merge with researchers_w_3_docs
researchers_w_3_docs_and_coded = researchers_coded_df.merge(
    researchers_w_3_docs,
    left_on=["researcher_id", "document_id"],
    right_on=["researcher_id", "document_id"],
)

def _mode_or_recent_reduce(group: pd.DataFrame, col: str) -> str:
    # Get the mode
    mode = group[col].mode().tolist()
    if len(mode) == 1:
        return mode[0]

    # Otherwise, iter over most recent publications until value in mode is found
    group_ordered_by_date = group.sort_values("publication_date", ascending=False)
    for _, row in group_ordered_by_date.iterrows():
        if row[col] in mode:
            return row[col]


def _agg_apply(group: pd.DataFrame) -> dict:
    return {
        "n_documents": group["document_id"].nunique(),
        "n_coded": group["coded_on_article"].sum(),
        "works_count": group["works_count"].iloc[0],
        "cited_by_count": group["cited_by_count"].iloc[0],
        "h_index": group["h_index"].iloc[0],
        "i10_index": group["i10_index"].iloc[0],
        "two_year_mean_citedness": group["two_year_mean_citedness"].iloc[0],
        "position": _mode_or_recent_reduce(group, "position"),
        "domain_name": _mode_or_recent_reduce(group, "domain_name"),
        "reduced_doc_type": _mode_or_recent_reduce(group, "reduced_doc_type"),
    }


researchers_w_3_docs_and_coded_agg = (
    researchers_w_3_docs_and_coded.groupby("researcher_id")[
        [col for col in researchers_w_3_docs_and_coded if col != "researcher_id"]
    ]
    .apply(_agg_apply)
    .reset_index(name="dicts")
)
researchers_w_3_docs_and_coded_agg = researchers_w_3_docs_and_coded_agg.join(
    pd.json_normalize(researchers_w_3_docs_and_coded_agg["dicts"])
).drop(columns=["dicts"])

# Create three features for coding status
# "any" coding status
# "majority" coding status
# "always" coding status
# determine type by taking the percentage of documents coded on
# and determining if it is greater than 0, greater than 0.5, or 1
researchers_w_3_docs_and_coded_agg["coding_pct"] = (
    researchers_w_3_docs_and_coded_agg["n_coded"]
    / researchers_w_3_docs_and_coded_agg["n_documents"]
)

researchers_w_3_docs_and_coded_agg["any_coding"] = (
    (researchers_w_3_docs_and_coded_agg["coding_pct"] > 0)
    & (researchers_w_3_docs_and_coded_agg["coding_pct"] <= 0.5)
).astype(int)
researchers_w_3_docs_and_coded_agg["majority_coding"] = (
    (researchers_w_3_docs_and_coded_agg["coding_pct"] > 0.5)
    & (researchers_w_3_docs_and_coded_agg["coding_pct"] < 1)
).astype(int)
researchers_w_3_docs_and_coded_agg["always_coding"] = (
    researchers_w_3_docs_and_coded_agg["coding_pct"] == 1
).astype(int)

# Drop n_documents, n_coded and
# rename "position" to "common_author_position",
# "domain_name" to "common_domain",
# and "reduced_doc_type" to "common_article_type"
researchers_w_3_docs_and_coded_agg = researchers_w_3_docs_and_coded_agg.drop(
    columns=["n_documents", "n_coded"]
)
researchers_w_3_docs_and_coded_agg = researchers_w_3_docs_and_coded_agg.rename(
    columns={
        "position": "common_author_position",
        "domain_name": "common_domain",
        "reduced_doc_type": "common_article_type",
    }
)

# Get dummies for categorical variables
researchers_w_3_docs_and_coded_agg_dummies = pd.get_dummies(
    researchers_w_3_docs_and_coded_agg,
    columns=["common_author_position", "common_domain", "common_article_type"],
    drop_first=True,
)

# Cast all to float
researchers_w_3_docs_and_coded_agg_dummies = (
    researchers_w_3_docs_and_coded_agg_dummies.astype(float)
)

Building upon previous work examining career implications for researchers who prioritize software development (see @sec-background), we investigated how varying levels of code contribution relate to scholarly impact through h-index metrics. To ensure a robust analysis, we applied several key data filtering steps. We only included researchers with at least three publications in our dataset, removed those with more than three developer account associations, and used each researcher's most common domain, article type, and author position, with ties broken by the most recent occurrence. We removed h-index outliers by excluding researchers below the bottom 3rd and above the top 97th percentiles. Finally, we removed any author-developer-account pairs with a predictive model confidence of less than 0.97. @tbl-h-index-counts summarizes the number of researchers in each coding frequency group, categorized by author position, publication type, and research domain.

We categorized researchers' coding contributions into mutually exclusive groups: non-coders (no code contributions), any coding (code contribution in less than half of article-repository pairs), majority coding (code contribution in at least half, but not all, article-repository pairs), and always coding (code contribution in every article-repository pair).

@fig-author-h-index-by-coding-status shows the distribution of author h-indices across these coding frequency groups, grouped by author position, publication type, and research domain.


In [ ]:
#| label: fig-author-h-index-by-coding-status
#| fig-cap: 'Distribution of author h-index by coding frequency across three key publication factors. Results are grouped by each author''s most frequent: (1) position in publication bylines (first, middle, or last), (2) publication type (preprint, research article, or software article), and (3) research domain (Social Sciences, Physical Sciences, Health Sciences, or Life Sciences). Within each subplot, h-indices are divided by the author''s coding frequency: ''none'' (no coding in any of their publications), ''any'' (coding in at least one but fewer than half of their publications), ''majority'' (coding in at least half but not all of their publications), and ''always'' (coding in each of their publications). Authors are only included if they have three or more publications within our dataset and are associated with no more than three developer accounts, with each association having a predicted model confidence of at least 97%.'

def _get_coding_status_string(row: pd.Series) -> str:
    if row["any_coding"] == 1:
        return "any"
    elif row["majority_coding"] == 1:
        return "majority"
    elif row["always_coding"] == 1:
        return "always"
    else:
        return "none"

researchers_w_3_docs_and_coded_agg["coding_status"] = researchers_w_3_docs_and_coded_agg.apply(
    _get_coding_status_string,
    axis=1,
)

# Select columns of interest
researchers_w_3_docs_and_coded_agg_subset_for_vis = researchers_w_3_docs_and_coded_agg[
    [
        "researcher_id",
        "h_index",
        "coding_status",
        "common_author_position",
        "common_domain",
        "common_article_type",
    ]
]

# Melt control variables
researchers_w_3_docs_and_coded_agg_vis_melted = researchers_w_3_docs_and_coded_agg_subset_for_vis.melt(
    id_vars=["researcher_id", "h_index", "coding_status"],
    value_vars=[
        "common_author_position",
        "common_domain",
        "common_article_type",
    ],
    var_name="control_var",
    value_name="control_value",
)

# Define visualization order and control variables
vis_order = ["none", "any", "majority", "always"]
control_vars = OrderedDict([
    ("common_author_position", ["first", "middle", "last"]),
    ("common_article_type", ["preprint", "research article", "software article"]),
    ("common_domain", ["Social Sciences", "Physical Sciences", "Health Sciences", "Life Sciences"]),
])

# Set up the figure with appropriate size
fig = plt.figure(figsize=(8, 9))

# Calculate grid dimensions based on control variables
num_controls = len(control_vars)
max_subsets = max(len(values) for values in control_vars.values())
grid_height = num_controls * 3  # 3 rows per control variable

# Iterate through control variables to create subplots
for control_idx, (control_var, control_values) in enumerate(control_vars.items()):
    # Determine row position for this control variable
    row_pos = control_idx * 3
    
    # Create subplots for each control value
    for col_idx, control_value in enumerate(control_values):
        # Create the subplot
        ax = plt.subplot2grid(
            (grid_height, max_subsets),
            (row_pos, col_idx),
            rowspan=2,
            colspan=1
        )
        
        # Filter data for this control variable and value
        subset_data = researchers_w_3_docs_and_coded_agg_vis_melted[
            (researchers_w_3_docs_and_coded_agg_vis_melted["control_var"] == control_var) & 
            (researchers_w_3_docs_and_coded_agg_vis_melted["control_value"] == control_value)
        ]
        
        # Create boxplot with specified configuration
        sns.boxplot(
            data=subset_data,
            x="coding_status",
            y="h_index",
            hue="coding_status",
            hue_order=vis_order,
            order=vis_order,
            ax=ax,
            showfliers=False,
            legend=False,
        )

        # Set ylim to row specific values
        ax.set_ylim([-1, 115])
        
        # Set titles and labels
        ax.set_title(f"{control_value.title()}", fontsize=14)
        
        if col_idx == 0:
            ax.set_ylabel("h-index", fontsize=14)
        else:
            ax.set_ylabel("")
            
        # Never show xlabel as we use legend
        # if col_idx == 0:
        #     ax.set_xlabel("Coding Frequency", loc="left", fontproperties={"size": 16, "weight": "bold"})
        # else:
        ax.set_xlabel("")
            
        # Rotate x-tick labels for better readability
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

    # Add row header for this control variable
    fig.text(
        0.13, 
        0.95 - (control_idx * 0.299), 
        f"{control_var.replace('_', ' ').title().replace("Common", "").strip()}", 
        fontsize=16, 
        ha="left", 
        va="center", 
        fontweight="bold"
    )


# # Add a legend outside the plots
# handles = [plt.Rectangle((0,0), 1, 1, color=tuple(cmaps.bold[2:]._colors.tolist()[i])) for i in range(len(vis_order))]
# legend = fig.legend(
#     handles, 
#     vis_order, 
#     bbox_to_anchor=(0.92, 0.52),  # Position at bottom center
#     fontsize=16,
#     title="Author Coding Frequency",
#     title_fontproperties={"size": 16, "weight": "bold"},
#     alignment="left",
# )

# Adjust layout
plt.subplots_adjust(hspace=0.2, wspace=0.2, bottom=0.1)
plt.tight_layout(rect=[0.03, 0.06, 0.97, 0.95])

# Draw a horizontal lines at those coordinates
for y in [0.69, 0.39]:
    line = plt.Line2D([0.066, 0.944], [y,y], transform=fig.transFigure, color="black", linestyle="--")
    fig.add_artist(line)

# Show plot
plt.show()

Our analysis revealed a consistent and statistically significant negative relationship between code contribution frequency and h-index across multiple analytical controls. Our initial uncontrolled analysis (@tbl-researcher-coding-status-no-control) indicates increasingly adverse h-index effects as researcher coding frequency increases. Compared to non-coding authors, researchers were associated with progressively lower h-indices: occasional code contributors showed a \~27.3% lower h-index (p \< 0.001), majority code contributors demonstrated a \~53.5% lower h-index (p \< 0.001), and always coding authors exhibited a \~62.1% lower h-index (p \< 0.001).

When controlling for author position (@tbl-researcher-coding-status-author-position), we found a general pattern of reduced h-indices with increased code contribution, with one notable exception. Occasional coding first authors were associated with a \~14.9% higher h-index (p \< 0.001), while always coding first authors saw a \~21.6% reduction compared to non-coding first authors (p \< 0.001). For middle and last authors, the pattern was more consistently negative. Middle authors who occasionally coded showed a \~26.6% lower h-index (p \< 0.001), and those always coding demonstrated a \~52.9% lower h-index (p \< 0.001). Similarly, last authors who occasionally coded experienced a \~13.1% lower h-index (p \< 0.001), with always coding authors showing a \~45.7% lower h-index (p \< 0.001).

When controlling for research domain (@tbl-researcher-coding-status-domain), majority coding scientists showed significant h-index reductions across all domains. Health sciences researchers saw the most dramatic reduction at \~76.5% (p \< 0.001), followed by physical sciences at \~52.6% (p \< 0.001), social sciences at \~51.4% (p \< 0.001), and life sciences at \~47.1% (p \< 0.001).

Analyzing by common article type (@tbl-researcher-coding-status-article-type) revealed similar patterns. For authors primarily publishing preprints, the h-index reductions were substantial: \~25.6% for occasional coding, \~53.5% for majority coding, and \~62.9% for always coding authors. Authors primarily publishing software articles showed slightly better but still significant reductions: \~33.1% for majority coding and \~33.0% for always coding authors.

Based on these findings, we ***accept*** our hypothesis (*H3*) that "the frequency with which individual researchers contribute code to their research projects is negatively correlated with their h-index." Our analysis demonstrates a clear and statistically significant negative relationship between coding frequency and scholarly impact as measured by the researcher's h-index. This relationship was robust across multiple analytical controls, including author position, research domain, and article type. These results are particularly striking because each of our models includes publication count as an input feature, suggesting that these h-index reductions persist even when accounting for total research output.

# Discussion

Our analysis reveals significant disparities in the recognition of software contributions to scientific research, with nearly 30% of articles having non-author code-contributors who received no formal authorship credit. This persistent pattern suggests a systemic disconnect between software development and scientific recognition systems, reflecting challenges in how scientific contributions are valued and credited. This exclusion reflects what @shapin1989invisible observed about scientific authority—the selective attribution of technical work as either genuine knowledge or mere skill significantly impacts who receives formal recognition. These findings further support previous research by @olivier_philippe_2019_2585783 and @Carver2022ASO documenting the frequent relegation of software contributors to either acknowledgment sections or receiving no credit at all, rather than authorship positions, despite the increasingly central role of software in scientific inquiry. The stability of this pattern over time indicates that this phenomenon has embedded itself in scientific software development rather than representing a transitional phase, raising questions about scientific labor and how reward structures integrate technical contributions.

Our finding that, on average, article-repository pairs have only a single code contributor mirrors prior work from @farber2020analyzingGithubPapers. Further, the distribution of code contributions across author positions provides context to the hierarchical organization of scientific work. First authors emerge as significantly more likely to contribute code with `{python} percent_first_authors_code_contributors_str` of all first authors in our dataset contributing code. Middle and last authors, meanwhile, were statistically significantly less likely to contribute code, with only `{python} percent_middle_authors_code_contributors_str` of middle authors and `{python} percent_last_authors_code_contributors_str` of last authors acting as code-contributing members of the research team. Corresponding authors were similarly less likely than expected to be code contributors, as we found that within our dataset, corresponding authors were code contributors `{python} percent_corresponding_authors_code_contributors_str` of the time. These patterns align with traditional scientific labor distribution where first authors typically handle technical aspects of research while middle and last authors are likely specialist contributors or provide guidance and oversight [@Larivire2020InvestigatingTD; @sauermann2017authorship]. However, our data did not support our initial hypothesis that corresponding authors would also be more likely to contribute code due to their shared responsibility for the long-term maintenance of research artifacts. This finding suggests a potential strict division between project management responsibilities and direct technical engagement with software development.

The modest citation advantage associated with code-contributing authors (5.1% increase in citations per code-contributing author) stands in contrast with the significant negative relationship between coding frequency and an individual's scholarly impact (h-index). This misalignment between technical contributions and scientific recognition creates an asymmetrical relationship in which software development may enhance research impact but potentially penalizes individual careers. The progressive reduction in h-index as coding frequency increases indicates a cumulative disadvantage for frequent code contributors. This pattern persists even when controlling for publication count, suggesting issues in how software contributions are valued relative to other scientific outputs. These findings echo concerns raised by @muna2016astropyproblem about the sustainability of research software development and highlight how current reward structures may discourage talented developers from pursuing scientific careers.

Software development represents a form of scholarly labor that has become increasingly essential to modern research yet remains incompletely integrated into formal recognition systems. Similar to the high proportion of articles with authors who made data curation contributions towards research observed by @Larivire2020InvestigatingTD, our finding that a quarter of papers have unacknowledged code contributors highlights a labor role that is simultaneously common and undervalued. The prevalence of code contributions across domains demonstrates the importance of this work to contemporary research. However, the persistent exclusion of contributors from authorship suggests that researchers continue to classify software development as technical support rather than intellectual contribution. This classification may reflect disciplinary traditions that privilege certain forms of scholarly production despite the growing recognition that software represents a legitimate research output [@Katz2020RecognizingTV]. The tension between software's importance and contributors' recognition status raises questions about how we define, value, and reward different forms of scientific labor in an increasingly computational research landscape.

## Limitations

Our data collection approach introduces several methodological constraints that should be considered when interpreting these results. By focusing exclusively on GitHub repositories, we likely miss contributions stored on alternative platforms such as GitLab, Bitbucket, or institutional repositories, potentially skewing our understanding of contribution patterns. As @trujillo2022penumbra, @Cao2023TheRO, and @escamilla2022riseofgithub have all noted, while GitHub is the predominate host of scientific software, significant portions of research code exist on other platforms. Additionally, our reliance on public repositories means we cannot account for private repositories or code that were never publicly shared, potentially underrepresenting sensitive research areas or proprietary methods.

Our predictive modeling approach for matching authors with developer accounts presents additional limitations. The model's performance can be affected by shorter names where less textual information is available for matching, potentially creating biases against researchers from cultures with shorter naming conventions. Organization accounts used for project management pose particular challenges for accurate matching, and while we implemented filtering mechanisms to minimize their impact, some misclassifications may persist. Furthermore, our approach may not capture all code contributors if multiple individuals developed code. However, only one uploaded it to a repository, creating attribution artifacts that may systematically underrepresent specific contributors, particularly junior researchers or technical staff who may not have direct repository access.

Our analytical approach required substantial data filtering to ensure reliable results, introducing potential selection biases in our sample. By focusing on article-repository pairs with commit activity no later than 90 days past the date of article publication and at least three authors and less than `{python} int(n_authors_97th_percentile)` authors, we may have systematically excluded certain types of research projects, particularly those with extended development timelines or extensive collaborations. Our categorization of coding status (non-coder, any coding, majority coding, always coding) necessarily simplifies complex contribution patterns. It does not account for code contributions' quality, complexity, or significance. Additionally, our reliance on OpenAlex metadata introduces certain limitations to our analysis. While OpenAlex provides good overall coverage, it lags behind proprietary databases in indexing references and citations. The lag in OpenAlex data may affect our citation-based analyses and the completeness of author metadata used in our study [@alperin2024analysis].

## Future Work

Future technical improvements may enhance our understanding of the relationship between software development and scientific recognition systems. Expanding analysis beyond GitHub to include other code hosting platforms would provide a more comprehensive understanding of scientific software development practices across domains and institutional contexts. More sophisticated entity-matching techniques could improve author-developer account identification, particularly for cases with limited information or common names. Developing more nuanced measures and classifications of code contribution type, quality, and significance beyond binary contribution identification would better capture the true impact of technical contributions to research. These methodological advances would enable more precise tracking of how code contributions translate—or fail to translate—into formal scientific recognition, providing clearer evidence for policy interventions.

Our findings point to several directions for future research on the changing nature of scientific labor and recognition. Longitudinal studies tracking how code contribution patterns affect career trajectories would provide valuable insights into the long-term impacts of the observed h-index disparities and whether these effects vary across career stages. Comparative analyses across different scientific domains could reveal discipline-specific norms and practices around software recognition, potentially identifying models that more equitably credit technical contributions. Qualitative studies examining how research teams make authorship decisions regarding code contributors would complement our quantitative findings by illuminating the social and organizational factors influencing recognition practices. Additionally, to better understand corresponding authors' role in maintaining research artifacts, future work could remove the 90-day post-publication commit activity filter to examine long-term sustainability actions. However, this approach would need to address the introduction of contributors unrelated to the original paper.

Despite their growing importance, the persistent underrecognition of software contributions suggests a need for structural interventions in how we conceptualize and reward scientific work. Building upon efforts like CRediT [@brand2015beyond], future work should investigate potential policy changes to better align institutional incentives with the diverse spectrum of contributions that drive modern scientific progress. However, as the example of CRediT demonstrates, even well-intentioned taxonomies may reproduce existing hierarchies or create new forms of inequality if they fail to address underlying power dynamics in scientific communities. The challenge is not merely technical but social: creating recognition systems that simultaneously support innovation, ensure appropriate credit, maintain research integrity, and foster equitable participation in an increasingly computational scientific enterprise.

# References

::: {#refs}
:::

# Appendix

## Extended Data and Methods

### Building a Dataset of Linked Scientific Articles and Code Repositories

The increasing emphasis on research transparency has led many journals and platforms to require or recommend code and data sharing [@stodden2013toward; @sharma2024analytical], creating traceable links between publications and code. These explicit links enable systematic study of both article-repository and author-developer account relationships [@Hata2021ScienceSoftwareLT; @Kelley2021AFF; @Stankovski2024RepoFromPaperAA; @milewicz2019characterizing].

Our dataset collection process leveraged four sources of linked scientific articles and code repositories, each with specific mechanisms for establishing these connections:

1.  **Public Library of Science (PLOS)**: We extracted repository links from PLOS articles' mandatory data and code availability statements.
2.  **Journal of Open Source Software (JOSS)**: JOSS requires explicit code repository submission and review as a core part of its publication process.
3.  **SoftwareX**: Similar to JOSS, SoftwareX mandates code repositories as a publication requirement.
4.  **Papers with Code**: This platform directly connects machine learning preprints with their implementations. We focus solely on the "official" article-repository relationships rather than the "unverified" or "unofficial" links.

To create a comprehensive and analyzable dataset, we enriched these article-repository pairs with metadata from multiple sources. We utilized the Semantic Scholar API for DOI resolution to ensure we found the latest version of each article. This resolution step was particularly important when working with preprints, as journals may have published these papers since their inclusion in the Papers with Code dataset. Using Semantic Scholar, we successfully resolved `{python} round(overall_doi_resolution * 100, 1)`% of all DOIs within our dataset[^4].

[^4]: Broken out by dataset source, we resolved `{python} round(per_dataset_doi_resolution["plos"] * 100, 1)`% of all PLOS DOIs, `{python} round(per_dataset_doi_resolution["joss"] * 100, 1)`% of all JOSS DOIs, `{python} round(per_dataset_doi_resolution["softwarex"] * 100, 1)`% of all SoftwareX DOIs, and `{python} round(per_dataset_doi_resolution["pwc"] * 100, 1)`% of all Papers with Code (arXiv) DOIs.

We then utilized the OpenAlex API to gather detailed publication metadata, including:

-   Publication characteristics (open access status, domain, publication date)
-   Author details (name, author position, corresponding author status)
-   Article- and individual-level metrics (citation counts, FWCI, h-index)

Similarly, the GitHub API provided comprehensive information for source code repositories:

-   Repository metadata (name, description, programming languages, creation date)
-   Contributor details (username, display name, email)
-   Repository-level metrics (star count, fork count, issue count)

### Developing a Predictive Model for Author-Developer Account Matching

#### Annotated Dataset Creation


In [ ]:
#| output: false

# We run this in it's own cell so that output from data download doesn't show
# Load annotated dataset
annotated_dataset = sci_soft_models_data.load_annotated_dev_author_em_dataset()

# Get counts of positive and negative examples
n_positive_examples = len(annotated_dataset.loc[annotated_dataset["match"]])
n_negative_examples = len(annotated_dataset.loc[~annotated_dataset["match"]])
pct_positive_examples = round(n_positive_examples / len(annotated_dataset), 3) * 100
pct_negative_examples = round(n_negative_examples / len(annotated_dataset), 3) * 100

# Create function to split row details
def split_dev_details(dev_details: str) -> dict:
    details_list = dev_details.split(";\\n")
    details_dict = {}
    for detail in details_list:
        try:
            key, value = detail.split(": ", 1)
        except Exception:
            print(detail)
            raise

        # Only keep username, name, and email
        if key in ["username", "name", "email"]:
            # If value is "None" replace with None
            if value == "None":
                value = None

            # Store to details dict
            details_dict[key] = value
    
    return details_dict

# Split dev details
all_dev_details = annotated_dataset["dev_details"].apply(split_dev_details)

# Convert to dataframe
all_dev_details_df = pd.DataFrame(all_dev_details.tolist())

# Drop duplicates on username
all_dev_details_df = all_dev_details_df.drop_duplicates(subset=["username"])

# Get counts
n_devs = len(all_dev_details_df)
n_devs_with_name = all_dev_details_df["name"].notnull().sum()
n_devs_with_email = all_dev_details_df["email"].notnull().sum()

pct_devs_with_name = round(n_devs_with_name / n_devs, 3) * 100
pct_devs_with_email = round(n_devs_with_email / n_devs, 3) * 100

# Get unique authors from original frame from unique semantic scholar id
n_authors = annotated_dataset["semantic_scholar_id"].nunique()

Creating an accurate author-developer account matching model required quality-labeled training data that reflects real-world identity matching challenges. Exact-matching on names or emails proved insufficient due to variations in formatting (e.g., "J. Doe" vs. "Jane Doe"), use of institutional versus personal email addresses, and incomplete information. However, author and developer account information often contains sufficient similarities for probabilistic matching, such as when author "Jane Doe" corresponds to username "jdoe" or "janedoe123".

To efficiently build our training and evaluation dataset, we used JOSS articles as we believed they typically feature higher author-developer-account overlap, increasing positive match density. Our dataset creation process followed these steps:

1.  We generated semantic embeddings for each developer account and author name using the [multi-qa-MiniLM-L6-cos-v1](https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1) model from the Sentence Transformers Python library [@reimers-2019-sentence-bert].
2.  We calculated cosine similarity between all potential author-developer-account pairs for each article-repository pair.
3.  For annotation efficiency, we selected the three most similar authors for each developer account.

From these generated author-developer-account pairs, we randomly selected 3,000 for classification by two independent annotators as either matches or non-matches, resolving disagreements through discussion and verification. The resulting dataset contains `{python} n_positive_examples` (`{python} pct_positive_examples`%) positive matches and `{python} n_negative_examples` (`{python} pct_negative_examples`%) negative matches, comprising `{python} n_authors` unique authors and `{python} n_devs` unique developer accounts.

Our collected data for annotation confirmed that exact matching would be insufficient—only `{python} int(n_devs_with_name)` (`{python} float(pct_devs_with_name)`%) of developer accounts had associated display names and just `{python} int(n_devs_with_email)` (`{python} float(pct_devs_with_email)`%) had associated email addresses.

#### Training and Evaluation {#sec-appendix-model-training-eval}


In [ ]:
training_split_counts = sci_soft_models_data.load_final_model_training_split_details()

# Take sum on rows
training_split_counts = training_split_counts.set_index("split").sum(axis=1)
training_split_pct = ((training_split_counts / training_split_counts.sum()) * 100).round(1)

Our training and evaluation methodology began with careful dataset preparation to prevent data leakage between training and test sets. To ensure complete separation of authors and developers, we randomly selected 10% of unique authors and 10% of unique developers, designating any pairs containing these selected entities for the test set. This entity-based splitting strategy resulted in `{python} int(training_split_counts["train"])` (`{python} float(training_split_pct["train"])`%) pairs for training and `{python} int(training_split_counts["test"])` (`{python} float(training_split_pct["test"])`%) pairs for testing.

For our predictive model, we evaluated three transformer-based architectures that have demonstrated strong performance in entity matching tasks:

-   DeBERTa-v3-base [@he2021debertav3; @he2021deberta]
-   mBERT (bert-base-multilingual-cased) [@bert2018]
-   DistilBERT [@Sanh2019DistilBERTAD]

We systematically evaluated these base models across different combinations of developer-account features, ranging from using only the username to incorporating complete profile information (username, display name, and email address). We fine-tuned all models using the Adam optimizer with a linear learning rate of 1e-05 for training and a batch size of 8 for training and evaluation. Given the size of our dataset and the binary nature of our classification task, models were trained for a single epoch to prevent overfitting.

We evaluated model performance using standard binary classification metrics: precision, recall, and F1-score. This evaluation framework allowed us to directly compare model architectures and feature combinations while accounting for the balance between precision and recall in identifying correct matches.

Our comprehensive model evaluation revealed that fine-tuning DeBERTa-v3-base [@he2021debertav3] with developer username and display name as input features produces optimal performance for author-developer matching. This model configuration achieved a binary F1 score of `{python} float(best_model_f1)`, with an accuracy of `{python} float(best_model_acc)`, precision of `{python} float(best_model_prec)`, and recall of `{python} float(best_model_rec)`. @tbl-em-model-comparison presents a complete comparison of model architectures and feature combinations.

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-em-model-comparison
#| tbl-cap: Comparison of Models for Author-Developer-Account Matching

exp_results

:::

::: {.content-visible when-format="pdf"}

In [ ]:
def df_to_latex_table(
    df: pd.DataFrame,
    caption: str,
    label: str,
    alignment: str = None,
    float_format: str = ".3f"
) -> str:
    # Generate column alignment if not provided
    if alignment is None:
        # Use 'l' for string/object columns, 'r' for numeric columns
        alignment = "".join(
            ["l" if dtype.kind in "OUS" else "r" for dtype in df.dtypes]
        )
    else:
        # Ensure provided alignment matches number of columns
        if len(alignment) != len(df.columns):
            raise ValueError(f"Alignment string length ({len(alignment)}) must match number of columns ({len(df.columns)})")
    
    # Create header row
    header_row = " & ".join([f"\\textbf{{{col}}}" for col in df.columns]) + " \\\\"
    
    # Create data rows with alternating colors
    data_rows = []
    for i, row in enumerate(df.itertuples(index=False)):
        color = "gray!10" if i % 2 == 1 else "white"
        
        # Format each cell value
        cells = []
        for val, dtype in zip(row, df.dtypes):
            if pd.isna(val):
                cells.append("")
            elif dtype.kind in "fc":  # Float or complex
                try:
                    cells.append(f"{float(val):{float_format}}")
                except (ValueError, TypeError):
                    cells.append(str(val))
            else:
                cells.append(str(val))
        
        # Create the row with consistent cell coloring
        colored_cells = [f"\\cellcolor{{{color}}}{cell}" for cell in cells]
        row_str = " & ".join(colored_cells) + " \\\\"
        data_rows.append(f"    {row_str}")
    
    # Combine into final table
    table_template = f"""\\begin{{table}}
\\centering
\\caption{{{caption}}}
\\label{{{label}}}
\\begin{{tabular}}{{{alignment}}}
\\toprule
{header_row}
\\midrule
{chr(10).join(data_rows)}
\\bottomrule
\\end{{tabular}}
\\end{{table}}"""

    return table_template

In [ ]:
# Convert to LaTeX table
model_comparison_latex = df_to_latex_table(
    exp_results,
    caption="Comparison of Models for Author-Developer-Account Matching",
    label="tbl-em-model-comparison",
    alignment="llrrrr",
)

IPython.display.Latex(model_comparison_latex)

:::

Analysis of each model's performance revealed that including developer display names had the largest positive impact on model performance compared to username alone. We also observed that mBERT's performance was comparable to DeBERTa's while using the developer email address as an additional input feature. However, we selected the DeBERTa configuration as it had consistent strong performance across various feature combinations.

To facilitate the reuse of our work, we have made our trained model and supporting code publicly available. Complete fine-tuning, evaluation, and inference code is available as the Python package: [sci-soft-models](https://github.com/evamaxfield/sci-soft-models), and the fine-tuned model has been released on HuggingFace ([evamxb/dev-author-em-clf](https://huggingface.co/evamxb/dev-author-em-clf)).

#### Model Limitations

While our model demonstrates strong performance, we acknowledge certain limitations in our approach:

1.  **Short name sensitivity**: Shorter names (both usernames and display names) can affect the model's performance, as less textual information is available for matching.
2.  **Organization accounts**: Research lab accounts used for project management present a potential challenge for accurate matching, as they don't correspond to individual authors. However, our filtering mechanisms applied before analysis help minimize their impact in modeling.

### Dataset Characteristics and Repository Types

Our compiled dataset appears to contain a mix of repository types, varying from analysis script repositories to software tools and likely some "code dumps" (where code is copied to a new repository immediately before publication). This diversity is reflected in the commit duration patterns across different publication types. The median commit duration for repositories in our analysis is:

-   47 days for preprints
-   104 days for research articles
-   247 days for software articles

Complete statistics on commit durations, including count, mean, and quantile details, are available in @tbl-commit-duration-distributions.


In [ ]:
#| label: tbl-commit-duration-distributions
#| tbl-cap: Commit duration (in days) distributions for different publication types. Only includes article-repository pairs with a most recent commit no later than 90 days after publication and excludes publications from research teams in the top 3% of total author sizes.

commit_dist_plotting_data = team_comp_no_push_after_pub.copy()
commit_dist_plotting_data = commit_dist_plotting_data.sort_values(
    "article_type",
)
commit_dist_plotting_data["repo_commit_duration"] = commit_dist_plotting_data["repo_commit_duration"] * 365.25
commit_dist_plotting_data.groupby(["article_type"])["repo_commit_duration"].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

## Distributions of Author-Developer-Account Prediction Confidence


In [ ]:
#| label: fig-dist-of-author-dev-pred-confidence
#| fig-cap: Distribution of author-developer-account prediction confidence scores. The left plot shows the distribution of all prediction confidence scores, while the right plot shows the distribution of prediction confidence scores for author-developer-account pairs with a confidence score greater than or equal to 0.97.

# Create a reduced copy of researcher_dev_links
reduced_researcher_dev_links = (
    researcher_dev_links.copy().sort_values(
        by="predictive_model_confidence",
        ascending=False,
    )
    .drop_duplicates(
        subset=["developer_account_id"],
        keep="first",
    )
    .drop_duplicates(
        subset=["researcher_id"],
        keep="first",
    )
)

# Select columns of interest, change column names, and merge
merged_reduced_researcher_dev_links = (
    reduced_researcher_dev_links[
        [
            "researcher_id",
            "developer_account_id",
            "predictive_model_confidence",
        ]
    ]
    .rename(
        columns={
            "predictive_model_confidence": "confidence",
        },
    )
    .merge(
        researchers.copy()[["id", "name"]].rename(
            columns={
                "id": "researcher_id",
                "name": "researcher_name",
            },
        ),
        on="researcher_id",
    )
    .merge(
        devs.copy()[["id", "username", "name"]].rename(
            columns={
                "id": "developer_account_id",
                "username": "dev_username",
                "name": "dev_provided_name",
            },
        ),
        on="developer_account_id",
    )
)[[
    "researcher_name",
    "dev_username",
    "dev_provided_name",
    "confidence",
]]

thresholded_reduced_researcher_dev_links = (
    merged_reduced_researcher_dev_links.copy().loc[
        merged_reduced_researcher_dev_links["confidence"] >= 0.97
    ]
)

# Caluclate how much data was removed
count_removed_from_conf_thresh = len(
    merged_reduced_researcher_dev_links
) - len(thresholded_reduced_researcher_dev_links)
pct_removed_from_conf_thresh = round(
    (
        1 - (len(thresholded_reduced_researcher_dev_links) / len(merged_reduced_researcher_dev_links))
    ) * 100,
    1,
)

# Add new column to "merged_reduced_researcher_dev_links" df that is "set" and has "full" as value
merged_reduced_researcher_dev_links["set"] = "full"
# Add new column to "thresholded_reduced_researcher_dev_links" df that is "set" and has "thresholded" as value
thresholded_reduced_researcher_dev_links["set"] = "threshold"

# Concatenate the two dataframes
full_and_thresh = pd.concat(
    [
        merged_reduced_researcher_dev_links,
        thresholded_reduced_researcher_dev_links,
    ],
    ignore_index=True,
).reset_index(drop=True)

# Create plot
g = sns.displot(
    full_and_thresh,
    x="confidence",
    kind="ecdf",
    col="set",
    hue="set",
    facet_kws={"sharex": False},
    legend=False,
)

# Set titles
g.set_titles(col_template="{col_name} set")
g.fig.tight_layout()

Thresholding the predictive model confidence at 0.97 resulted in a `{python} pct_removed_from_conf_thresh`% (`{python} count_removed_from_conf_thresh`) reduction in the number of author-developer-account pairs (from an unfiltered total of `{python} len(merged_reduced_researcher_dev_links)` author-developer-account pairs). This threshold was chosen to ensure a high level of confidence in the matches while retaining a large number of pairs for analysis.

## Filtered Dataset Description for Article-Citation, Author-Position, and Author-Correspondence Analysis

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-team-comp-no-push-after-pub-counts
#| tbl-cap: Counts of article-repository pairs, authors, and developers for research teams. Only includes research teams from article-repository pairs with a most recent commit no later than 90 days after publication and excludes research teams in the top 3% of total author sizes.

team_comp_no_push_after_pub_counts_html = _get_html_for_counts_table(
    domain_stats=team_comp_no_push_after_pub_counts_domain,
    doc_type_stats=team_comp_no_push_after_pub_counts_article_type,
    access_stats=team_comp_no_push_after_pub_counts_open_access,
    data_source_stats=team_comp_no_push_after_pub_counts_dataset_source,
    total_article_repo_pairs=team_comp_no_push_after_pub_total_pairs,
    total_authors=team_comp_no_push_after_pub_total_authors,
    total_devs=team_comp_no_push_after_pub_total_devs,
)

IPython.display.HTML(team_comp_no_push_after_pub_counts_html)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
team_comp_no_push_after_pub_counts_latex = _get_latex_for_counts_table(
    caption=(
        "Counts of article-repository pairs, authors, and developers for research teams. "
        "Only includes research teams from article-repository pairs with a "
        "most recent commit no later than 90 days after publication and excludes "
        "research teams in the top 3\% of total author sizes."
    ),
    label="tbl-team-comp-no-push-after-pub-counts",
    domain_stats=team_comp_no_push_after_pub_counts_domain,
    doc_type_stats=team_comp_no_push_after_pub_counts_article_type,
    access_stats=team_comp_no_push_after_pub_counts_open_access,
    data_source_stats=team_comp_no_push_after_pub_counts_dataset_source,
    total_article_repo_pairs=team_comp_no_push_after_pub_total_pairs,
    total_authors=team_comp_no_push_after_pub_total_authors,
    total_devs=team_comp_no_push_after_pub_total_devs,
)

IPython.display.Latex(team_comp_no_push_after_pub_counts_latex)

:::

## Article Citation Linear Model Results

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-article-composition-overall
#| tbl-cap: Article citations by code contributorship of research team. Generalized linear model fit with negative binomial distribution and log link function.

article_cited_by_count_models["no-control"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    article_cited_by_count_models["no-control"],
    tbl_cap=(
        "Article citations by code contributorship of research team. "
        "Generalized linear model fit with negative binomial distribution "
        "and log link function. Significant p-values are indicated with asterisks: "
        "p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)"
    ),
    tbl_label="tbl-article-composition-overall",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-article-composition-oa-status
#| tbl-cap: Article citations by code contributorship of research team controlled by open access status. Generalized linear model fit with negative binomial distribution and log link function.

article_cited_by_count_models["Is Open Access"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    article_cited_by_count_models["Is Open Access"],
    tbl_cap=(
        "Article citations by code contributorship of research team "
        "controlled by open access status. "
        "Generalized linear model fit with negative binomial distribution "
        "and log link function. Significant p-values are indicated with asterisks: "
        "p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)"
    ),
    tbl_label="tbl-article-composition-oa-status",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-article-composition-domain
#| tbl-cap: Article citations by code contributorship of research team controlled by domain. Generalized linear model fit with negative binomial distribution and log link function.

article_cited_by_count_models["Domain"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    article_cited_by_count_models["Domain"],
    tbl_cap=(
        "Article citations by code contributorship of research team "
        "controlled by domain. "
        "Generalized linear model fit with negative binomial distribution "
        "and log link function. Significant p-values are indicated with asterisks: "
        "p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)"
    ),
    tbl_label="tbl-article-composition-domain",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-article-composition-type
#| tbl-cap: Article citations by code contributorship of research team controlled by article type. Generalized linear model fit with negative binomial distribution and log link function.

article_cited_by_count_models["Article Type"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    article_cited_by_count_models["Article Type"],
    tbl_cap=(
        "Article citations by code contributorship of research team "
        "controlled by article type. "
        "Generalized linear model fit with negative binomial distribution "
        "and log link function. Significant p-values are indicated with asterisks: "
        "p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)"
    ),
    tbl_label="tbl-article-composition-type",
)

IPython.display.Latex(model_results_latex)

:::

## Post-Hoc Tests for Coding vs Non-Coding Authors by Position

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-post-hoc-tests-on-author-positions
#| tbl-cap: 'Counts of Code-Contributing Authors (''Coding'') as well as Total Authors by Position and Bonferroni Corrected p-values from Post-Hoc Binomial Tests. Significant p-values are indicated with asterisks: p < 0.05 (`*`), p < 0.01 (`**`), p < 0.001 (`***`).'

def create_multirow_html_table(
    dataframes_dict: dict[str, pd.DataFrame],
    split_var: str,
):
    # HTML templates for rows
    initial_row_for_control_template = """
    <tr>
        <td rowspan="{n_control_rows}"><b>{control}</b></td>
        <td rowspan="{n_subset_rows}"><b>{subset}</b></td>
        <td>{split_val}</td>
        <td>{coding}</td>
        <td>{total}</td>
        <td>{p_value}</td>
    </tr>
    """.strip()

    initial_row_for_subset_template = """
    <tr>
        <td rowspan="{n_subset_rows}"><b>{subset}</b></td>
        <td>{split_val}</td>
        <td>{coding}</td>
        <td>{total}</td>
        <td>{p_value}</td>
    </tr>
    """.strip()

    subsequent_row_template = """
    <tr>
        <td>{split_val}</td>
        <td>{coding}</td>
        <td>{total}</td>
        <td>{p_value}</td>
    </tr>
    """.strip()

    # Generate table rows
    all_rows = []
    
    for control, df in dataframes_dict.items():
        rows_for_control = []
        # Group by subset to handle the spans
        for subset_name, subset_group in df.groupby("Subset"):
            subset_rows = []
            for idx, row in subset_group.iterrows():
                # Format p-value
                p_value = f"{row['p']:.3f}" if row['p'] >= 0.001 else "0.000"
                if row["p"] < 0.001:
                    p_value = f"{p_value}***"
                elif row["p"] < 0.01:
                    p_value = f"{p_value}**"
                elif row["p"] < 0.05:
                    p_value = f"{p_value}*"
                
                row_data = {
                    "control": control,
                    "subset": subset_name,
                    "split_val": row[split_var],
                    "coding": row["Coding"],
                    "total": row["Total"],
                    "p_value": p_value,
                    "n_subset_rows": len(subset_group),
                    "n_control_rows": len(df),
                }
                
                if len(rows_for_control) == 0 and len(subset_rows) == 0:
                    subset_rows.append(initial_row_for_control_template.format(**row_data))
                elif len(subset_rows) == 0:
                    subset_rows.append(initial_row_for_subset_template.format(**row_data))
                else:
                    subset_rows.append(subsequent_row_template.format(**row_data))
            
            rows_for_control.extend(subset_rows)

        all_rows.extend(rows_for_control)

    # Create the final HTML table
    table_html = f"""
    <table>
        <tr>
            <th><b>Control</b></th>
            <th><b>Subset</b></th>
            <th><b>{split_var.replace('_', ' ').title()}</b></th>
            <th><b>Coding</b></th>
            <th><b>Total</b></th>
            <th><b>p</b></th>
        </tr>
        {'\n'.join(all_rows)}
    </table>
    """.strip()
    
    return table_html

IPython.display.HTML(
    create_multirow_html_table(
        {
            "Domain": domain_code_char,
            "Article Type": article_type_code_char,
            "Open Access Status": open_access_status_code_char,
            "Overall": overall_code_char,
        },
        split_var="Position",
    )
)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
def create_multirow_latex_table(
    dataframes_dict: dict[str, pd.DataFrame],
    caption: str,
    label: str,
    split_var: str,
):
    # LaTeX templates for rows
    initial_row_for_control_template = """    \\multirow{{{n_control_rows}}}{{*}}{{\\textbf{{{control}}}}} & \\multirow{{{n_subset_rows}}}{{*}}{{\\textbf{{{subset}}}}} & {color}{split_val} & {color}{coding} & {color}{total} & {color}{p_value} \\\\"""

    initial_row_for_subset_template = """    & \\multirow{{{n_subset_rows}}}{{*}}{{\\textbf{{{subset}}}}} & {color}{split_val} & {color}{coding} & {color}{total} & {color}{p_value} \\\\"""

    subsequent_row_template = """    & & {color}{split_val} & {color}{coding} & {color}{total} & {color}{p_value} \\\\"""

    # Table header and footer templates
    table_header = """\\begin{table}
\\centering
\\small
\\caption{%s}
\\label{%s}
\\begin{tabular}{llllrr}
\\toprule
\\textbf{Control} & \\textbf{Subset} & \\textbf{%s} & \\textbf{Coding} & \\textbf{Total} & \\textbf{p} \\\\
\\midrule""" % (caption, label, split_var.replace('_', ' ').title())

    table_footer = """\\bottomrule
\\end{tabular}
\\end{table}"""

    # Generate table rows
    all_rows = []
    overall_row_count = 0  # Counter for all rows to handle coloring
    
    for control, df in dataframes_dict.items():
        rows_for_control = []
        # Group by subset to handle the spans
        for subset_name, subset_group in df.groupby("Subset"):
            subset_rows = []
            for idx, row in subset_group.iterrows():
                # Format p-value with significance stars
                p_value = f"{row['p']:.3f}" if row['p'] >= 0.001 else "0.000"
                if row["p"] < 0.001:
                    p_value = f"{p_value}$^{{***}}$"
                elif row["p"] < 0.01:
                    p_value = f"{p_value}$^{{**}}$"
                elif row["p"] < 0.05:
                    p_value = f"{p_value}$^{{*}}$"
                
                # Add cell color for odd-numbered rows
                color = "\\cellcolor{gray!10}" if overall_row_count % 2 == 1 else ""
                
                row_data = {
                    "control": control,
                    "subset": subset_name,
                    "split_val": row[split_var],
                    "coding": row["Coding"],
                    "total": row["Total"],
                    "p_value": p_value,
                    "n_subset_rows": len(subset_group),
                    "n_control_rows": len(df),
                    "color": color
                }
                
                if len(rows_for_control) == 0 and len(subset_rows) == 0:
                    subset_rows.append(initial_row_for_control_template.format(**row_data))
                elif len(subset_rows) == 0:
                    subset_rows.append(initial_row_for_subset_template.format(**row_data))
                else:
                    subset_rows.append(subsequent_row_template.format(**row_data))
                
                overall_row_count += 1
            
            rows_for_control.extend(subset_rows)
            
        # Add midrule between controls (except after the last control)
        if control != list(dataframes_dict.keys())[-1]:
            rows_for_control.append("    \\midrule")
            
        all_rows.extend(rows_for_control)

    # Combine all parts
    latex_table = f"""{table_header}
{chr(10).join(all_rows)}
{table_footer}"""
    
    return latex_table

IPython.display.Latex(
    create_multirow_latex_table(
        {
            "Domain": domain_code_char,
            "Article Type": article_type_code_char,
            "Open Access Status": open_access_status_code_char,
            "Overall": overall_code_char,
        },
        caption="Counts of Code-Contributing Authors ('Coding') as well as Total Authors by Position and Bonferroni Corrected p-values from Post-Hoc Binomial Tests. Significant p-values are indicated with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***).",
        label="tbl-post-hoc-tests-on-author-positions",
        split_var="Position",
    )
)

:::

Counts of authors in @tbl-post-hoc-tests-on-author-positions may differ slightly from counts in @tbl-team-comp-no-push-after-pub-counts. @tbl-team-comp-no-push-after-pub-counts counts unique authors, while @tbl-post-hoc-tests-on-author-positions counts unique author-document pairs (i.e., the same author may appear in multiple documents).

## Post-Hoc Tests for Coding vs Non-Coding Authors by Corresponding Status

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-post-hoc-tests-on-corresponding-status
#| tbl-cap: 'Counts of Code-Contributing Authors (''Coding'') as well as Total Authors by Corresponding Status and Bonferroni Corrected p-values from Post-Hoc Binomial Tests. Significant p-values are indicated with asterisks: p < 0.05 (`*`), p < 0.01 (`**`), p < 0.001 (`***`).'

IPython.display.HTML(
    create_multirow_html_table(
        {
            "Domain": domain_code_corresponding,
            "Article Type": article_type_code_corresponding,
            "Open Access Status": open_access_status_code_corresponding,
            "Overall": overall_code_corresponding,
        },
        split_var="Is Corresponding",
    )
)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
IPython.display.Latex(
    create_multirow_latex_table(
        {
            "Domain": domain_code_corresponding,
            "Article Type": article_type_code_corresponding,
            "Open Access Status": open_access_status_code_corresponding,
            "Overall": overall_code_corresponding,
        },
        caption="Counts of Code-Contributing Authors ('Coding') as well as Total Authors by Corresponding Status and Bonferroni Corrected p-values from Post-Hoc Binomial Tests. Significant p-values are indicated with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***).",
        label="tbl-post-hoc-tests-on-corresponding-status",
        split_var="Is Corresponding",
    )
)

:::

Counts of authors in @tbl-post-hoc-tests-on-corresponding-status may differ slightly from counts in @tbl-team-comp-no-push-after-pub-counts. @tbl-team-comp-no-push-after-pub-counts counts unique authors, while @tbl-post-hoc-tests-on-corresponding-status counts unique author-document pairs (i.e., the same author may appear in multiple documents).

## Filtered Dataset Description for h-Index Analysis


In [ ]:
def _get_counts_by_column_for_h_index_data(
    df: pd.DataFrame,
    column_name: str,
) -> list[dict[str, int]]:
    counts = []
    for value in df[column_name].unique():
        subset = df.loc[df[column_name] == value]

        # Append counts
        counts.append({
            column_name: value,
            "n_authors": subset["researcher_id"].nunique(),
            "n_any_coding": int(subset["any_coding"].sum()),
            "n_majority_coding": int(subset["majority_coding"].sum()),
            "n_always_coding": int(subset["always_coding"].sum()),
        })

    return counts

h_index_domain_stats = _get_counts_by_column_for_h_index_data(
    researchers_w_3_docs_and_coded_agg,
    "common_domain",
)
h_index_article_type_stats = _get_counts_by_column_for_h_index_data(
    researchers_w_3_docs_and_coded_agg,
    "common_article_type",
)
h_index_author_position_stats = _get_counts_by_column_for_h_index_data(
    researchers_w_3_docs_and_coded_agg,
    "common_author_position",
)

h_index_total_authors = researchers_w_3_docs_and_coded_agg["researcher_id"].nunique()
h_index_n_any_coding = int(researchers_w_3_docs_and_coded_agg["any_coding"].sum())
h_index_n_majority_coding = int(researchers_w_3_docs_and_coded_agg["majority_coding"].sum())
h_index_n_always_coding = int(researchers_w_3_docs_and_coded_agg["always_coding"].sum())

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-h-index-counts
#| tbl-cap: Counts of Total Authors, n Any Coding Authors, n Majority Coding Authors, and n Always Coding Authors by Common Domain, Document Type, and Author Position. Authors are only included if they have three or more publications within our dataset and are associated with no more than three developer accounts, with each association having a predicted model confidence of at least 97%.

# Construct multi-row span HTML table
# Columns should be: "n_authors", "n_any_coding", "n_majority_coding", "n_always_coding"
# Rows should be:
# "By Common Domain", "By Common Document Type", "By Common Author Position" and "Total"

# HTML templates
stats_piece_inital_row_template = """
<tr>
<td rowspan="{n_rows}">{row_name}</td>
<td>{value_name}</td>
<td>{n_authors}</td>
<td>{n_any_coding}</td>
<td>{n_majority_coding}</td>
<td>{n_always_coding}</td>
</tr>
""".strip()

stats_piece_subsequent_row_template = """
<tr>
<td>{value_name}</td>
<td>{n_authors}</td>
<td>{n_any_coding}</td>
<td>{n_majority_coding}</td>
<td>{n_always_coding}</td>
</tr>
""".strip()

# Iter over stats portions (and total)
stats_portions_html = []
for stats_portion, stats_name, value_key in [
    (h_index_domain_stats, "<b>By Commmon Domain</b>", "common_domain"),
    (h_index_article_type_stats, "<b>By Document Type</b>", "common_article_type"),
    (h_index_author_position_stats, "<b>By Author Position</b>", "common_author_position"),
    (
        [
            {
                "empty": "",
                "n_authors": f"<b>{h_index_total_authors}</b>",
                "n_any_coding": f"<b>{h_index_n_any_coding}</b>",
                "n_majority_coding": f"<b>{h_index_n_majority_coding}</b>",
                "n_always_coding": f"<b>{h_index_n_always_coding}</b>",
            }
        ],
        "<b>Total</b>",
        "empty",
    ),
]:
    # Order by value_key
    if value_key != "empty":
        stats_portion = sorted(
            stats_portion, key=lambda x: x[value_key]
        )

    stats_portion_html = []
    for i, stats_piece in enumerate(stats_portion):
        if i == 0:
            stats_portion_html.append(
                stats_piece_inital_row_template.format(
                    n_rows=len(stats_portion),
                    row_name=stats_name,
                    value_name=stats_piece[value_key].title(),
                    n_authors=stats_piece["n_authors"],
                    n_any_coding=stats_piece["n_any_coding"],
                    n_majority_coding=stats_piece["n_majority_coding"],
                    n_always_coding=stats_piece["n_always_coding"],
                )
            )
        else:
            stats_portion_html.append(
                stats_piece_subsequent_row_template.format(
                    value_name=stats_piece[value_key].title(),
                    n_authors=stats_piece["n_authors"],
                    n_any_coding=stats_piece["n_any_coding"],
                    n_majority_coding=stats_piece["n_majority_coding"],
                    n_always_coding=stats_piece["n_always_coding"],
                )
            )

    stats_portions_html.append("\n".join(stats_portion_html))

# Concat and wrap in table
stats_table_html = f"""
<table>
<tr>
    <th><b>Category</b></th>
    <th><b>Subset</b></th>
    <th><b>Total Authors</b></th>
    <th><b>Any Code</b></th>
    <th><b>Majority Code</b></th>
    <th><b>Always Code</b></th>
</tr>
{" ".join(stats_portions_html)}
</table>
""".strip()

IPython.display.HTML(stats_table_html)

:::

::: {.content-visible when-format="pdf"}

In [ ]:
# LaTeX templates
stats_piece_initial_row_template = """    \\multirow{{{n_rows}}}{{*}}{{\\textbf{{{row_name}}}}} & \\cellcolor{{gray!10}}{value_name} & \\cellcolor{{gray!10}}{n_authors} & \\cellcolor{{gray!10}}{n_any_coding} & \\cellcolor{{gray!10}}{n_majority_coding} & \\cellcolor{{gray!10}}{n_always_coding} \\\\"""

stats_piece_subsequent_row_template = """    & {value_name} & {n_authors} & {n_any_coding} & {n_majority_coding} & {n_always_coding} \\\\"""

caption = (
    "Counts of Total Authors, n Any Coding Authors, n Majority Coding Authors, "
    "and n Always Coding Authors by Common Domain, Document Type, "
    "and Author Position. Authors are only included if they have three "
    "or more publications within our dataset and are associated with "
    "no more than three developer accounts, with each association "
    "having a predicted model confidence of at least 97\%."
)
label = "tbl-h-index-counts"

# Table header and footer templates
table_header_start = """\\begin{{table}}
\\centering
\\small
\\caption{{{caption}}}
\\label{{{label}}}
\\begin{{tabular}}{{llrrrr}}
\\toprule
\\textbf{{Category}} & \\textbf{{Subset}} & \\textbf{{Total Authors}} & \\textbf{{Any Code}} & \\textbf{{Majority Code}} & \\textbf{{Always Code}} \\\\
\\midrule""".format(
    caption=caption,
    label=label,
)

table_footer = """\\bottomrule
\\end{tabular}
\\end{table}"""

# Generate table content
stats_portions_latex = []
for stats_portion, stats_name, value_key in [
    (h_index_domain_stats, "By Commmon Domain", "common_domain"),
    (h_index_article_type_stats, "By Document Type", "common_article_type"),
    (h_index_author_position_stats, "By Author Position", "common_author_position"),
    (
        [
            {
                "empty": "",
                "n_authors": h_index_total_authors,
                "n_any_coding": h_index_n_any_coding,
                "n_majority_coding": h_index_n_majority_coding,
                "n_always_coding": h_index_n_always_coding,
            }
        ],
        "Total",
        "empty",
    ),
]:
    # Order by value_key
    if value_key != "empty":
        stats_portion = sorted(
            stats_portion, key=lambda x: x[value_key]
        )

    stats_portion_latex = []
    if stats_name == "Total":
        stats_portion_latex.append(
            f"    \\textbf{{Total}} & & \\textbf{{{stats_portion[0]['n_authors']}}} & \\textbf{{{stats_portion[0]['n_any_coding']}}} & \\textbf{{{stats_portion[0]['n_majority_coding']}}} & \\textbf{{{stats_portion[0]['n_always_coding']}}} \\\\"
        )
    else:
        # Add the first row with the category label and colored cells
        first_piece = stats_portion[0]
        stats_portion_latex.append(
            stats_piece_initial_row_template.format(
                n_rows=len(stats_portion),
                row_name=stats_name,
                value_name=first_piece[value_key].title(),
                n_authors=first_piece["n_authors"],
                n_any_coding=first_piece["n_any_coding"],
                n_majority_coding=first_piece["n_majority_coding"],
                n_always_coding=first_piece["n_always_coding"],
            ).rstrip()
        )
        
        # Add subsequent rows with alternating colors
        for i, stats_piece in enumerate(stats_portion[1:]):
            color_cmd = "\\cellcolor{gray!10}" if i % 2 == 1 else ""
            stats_portion_latex.append(
                f"    & {color_cmd}{stats_piece[value_key]} & {color_cmd}{stats_piece['n_authors']} & {color_cmd}{stats_piece['n_any_coding']} & {color_cmd}{stats_piece['n_majority_coding']} & {color_cmd}{stats_piece['n_always_coding']} \\\\"
            )

    section_latex = "\n".join(stats_portion_latex)
    if stats_name != "Total":
        section_latex += "\\midrule"
    stats_portions_latex.append(section_latex)

# Combine all parts
stats_table_latex = f"""{table_header_start}
{"\n".join(stats_portions_latex)}
{table_footer}"""

IPython.display.Latex(stats_table_latex)

:::

## h-Index Linear Model Results


In [ ]:
def compute_researcher_level_models(
    y_col: str,
    data: pd.DataFrame,
    glm_family: sm.families.Family,
) -> dict[str, sm.GLM]:
    # Remove all "zero" y_col authors
    no_outliers = data[data[y_col] > 0].copy()

    # Remove outliers
    no_outliers = no_outliers[
        no_outliers[y_col].between(
            no_outliers[y_col].quantile(0.03),
            no_outliers[y_col].quantile(0.97),
        )
    ].copy()

    # Replace names
    no_outliers = no_outliers.rename(
        columns={
            "works_count": "Works Count",
            "any_coding": "Any Coding",
            "majority_coding": "Majority Coding",
            "always_coding": "Always Coding",
        }
    )

    # Replace other names (except const.) by removing `_` and title casing
    no_outliers = no_outliers.rename(
        columns=lambda x: x.replace("_", " ").title() if x not in ["const", y_col] else x
    )

    # Common features to use in all models
    required_features = [
        y_col,
        "Works Count",
        "Any Coding",
        "Majority Coding",
        "Always Coding",
    ]

    # Iter over different control variables and create models for each
    models = {}
    for control_var in [
        "no-control",
        "Common Author Position",
        "Common Article Type",
        "Common Domain",
    ]:
        if control_var != "no-control":
            # Get control variable list
            control_variables = [
                col for col in no_outliers.columns if col.startswith(control_var)
            ]

            # Create control variable subset of the data
            control_var_subset = no_outliers[required_features + control_variables].copy()

            # Create interactions
            for coding_status_col in ["Any Coding", "Majority Coding", "Always Coding"]:
                for control_col in control_variables:
                    control_var_subset[f"{coding_status_col} × {control_col}"] = (
                        control_var_subset[coding_status_col]
                        * control_var_subset[control_col]
                    )
        else:
            control_var_subset = no_outliers[required_features].copy()

        # Drop inf and nan
        control_var_subset = control_var_subset.replace(
            [float("inf"), -float("inf")], float("nan")
        ).dropna()

        # Create x and y
        y = control_var_subset[y_col]
        x = control_var_subset.drop(columns=[y_col])
        x = sm.add_constant(x)

        # Fit model
        model = sm.GLM(y, x, family=glm_family).fit()
        models[control_var] = model

    return models

# Create models for h_index
author_h_index_models = compute_researcher_level_models(
    "h_index",
    researchers_w_3_docs_and_coded_agg_dummies,
    glm_family=sm.families.Gaussian(sm.families.links.Log()),
)

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-researcher-coding-status-no-control
#| tbl-cap: Code-contributing authors h-index by coding status. Generalized linear model fit with Gaussian distribution and log link function.

author_h_index_models["no-control"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    author_h_index_models["no-control"],
    tbl_cap=(
        "Code-contributing authors h-index by coding status. "
        "Generalized linear model fit with Gaussian distribution "
        "and log link function. Significant p-values are indicated "
        "with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)."
    ),
    tbl_label="tbl-researcher-coding-status-no-control",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-researcher-coding-status-author-position
#| tbl-cap: Code-contributing authors h-index by coding status controlled by most freq. author position. Generalized linear model fit with Gaussian distribution and log link function.

author_h_index_models["Common Author Position"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    author_h_index_models["Common Author Position"],
    tbl_cap=(
        "Code-contributing authors h-index by coding status "
        "controlled by most freq. author position. "
        "Generalized linear model fit with Gaussian distribution and log link function."
        "Significant p-values are indicated "
        "with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)."
    ),
    tbl_label="tbl-researcher-coding-status-author-position",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-researcher-coding-status-domain
#| tbl-cap: Code-contributing authors h-index by coding status controlled by most freq. domain. Generalized linear model fit with Gaussian distribution and log link function.

author_h_index_models["Common Domain"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    author_h_index_models["Common Domain"],
    tbl_cap=(
        "Code-contributing authors h-index by coding status "
        "controlled by most freq. domain. "
        "Generalized linear model fit with Gaussian distribution and log link function."
        "Significant p-values are indicated "
        "with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)."
    ),
    tbl_label="tbl-researcher-coding-status-domain",
)

IPython.display.Latex(model_results_latex)

:::

::: {.content-visible when-format="html"}

In [ ]:
#| label: tbl-researcher-coding-status-article-type
#| tbl-cap: Code-contributing authors h-index by coding status controlled by most freq. article type. Generalized linear model fit with Gaussian distribution and log link function.

author_h_index_models["Common Article Type"].summary()

:::

::: {.content-visible when-format="pdf"}

In [ ]:
model_results_latex, model_results_df = convert_model_results_to_printable_pdf_ready(
    author_h_index_models["Common Article Type"],
    tbl_cap=(
        "Code-contributing authors h-index by coding status "
        "controlled by most freq. article type. "
        "Generalized linear model fit with Gaussian distribution and log link function."
        "Significant p-values are indicated "
        "with asterisks: p < 0.05 (*), p < 0.01 (**), p < 0.001 (***)."
    ),
    tbl_label="tbl-researcher-coding-status-article-type",
)

IPython.display.Latex(model_results_latex)

:::

## Analysis of Project Duration and Percentage Code-Contributors Who Are Authors

In our pre-registered analysis plan (<https://osf.io/fc74m>), we originally hypothesized about the relationship between project duration and authorship recognition. Specifically, we posited that sustained technical engagement and scientific recognition might be meaningfully related, with longer project durations potentially leading to higher rates of code-contributor authorship. We saw repository histories as providing a unique opportunity to examine this relationship, leading us to hypothesize that projects with longer commit durations would be associated with higher percentages of developers receiving authorship recognition (pre-registered as H2).


In [ ]:
team_comp_no_push_after_pub_for_duration = team_comp_no_push_after_pub.copy()

# Calculate ratio
team_comp_no_push_after_pub_for_duration["pct_devs_authors"] = (
    team_comp_no_push_after_pub_for_duration["n_author_devs"]
    / (
        team_comp_no_push_after_pub_for_duration["n_author_devs"]
        + team_comp_no_push_after_pub_for_duration["n_non_author_devs"]
    )
)

# Drop inf and nan
team_comp_no_push_after_pub_for_duration = (
    team_comp_no_push_after_pub_for_duration.replace(
        [float("inf"), -float("inf")], float("nan")
    ).dropna()
)

# Remove negative "repo_commit_duration" repos
team_comp_no_push_after_pub_for_duration = team_comp_no_push_after_pub_for_duration[
    team_comp_no_push_after_pub_for_duration["repo_commit_duration"] >= 0
]

# Run pearson correlation
non_author_to_author_dev_pearson_results = pearsonr(
    team_comp_no_push_after_pub_for_duration["pct_devs_authors"],
    team_comp_no_push_after_pub_for_duration["repo_commit_duration"],
)

pearson_rho = f"{non_author_to_author_dev_pearson_results[0]:.2f}"

However, our analysis found no evidence to support this hypothesis. When examining the relationship between a repository's commit duration and the percentage of developers who receive authorship recognition, we found no significant correlation (r = `{python} pearson_rho`, p = n.s.). This suggests that the length of time a project has been in development has no meaningful relationship with the proportion of developers who are recognized as authors.

We ultimately decided to move this analysis to the appendix for two key methodological reasons. First, our approach of using repository-level commit duration as a proxy for individual contribution patterns proved too coarse-grained. A more precise analysis would need to examine individual-level contribution durations and patterns rather than overall project length. Second, our method did not account for the varying levels of contribution that different developers make to a repository. Simply correlating overall project duration with authorship rates fails to capture the nuanced ways that sustained, meaningful technical contributions might influence authorship decisions.

These limitations suggest potential directions for future work that could more rigorously examine the relationship between long-term technical engagement and scientific recognition. Such work might benefit from more granular analysis of individual contribution patterns, perhaps incorporating measures of contribution significance and sustainability rather than just temporal duration.